# Load API Files

## AWS Data

In [1]:
from datetime import datetime
from datetime import timedelta
import requests  # requests 모듈 임포트
import csv
import numpy as np
from io import StringIO
import pandas as pd
from time import sleep
import logging
import math

import urllib3

from statsmodels.tsa.stattools import adfuller

In [2]:
sleep(1)

In [3]:
def return_url_aws(): # Real-time data retrieval
    tm = datetime.now()
    tm = tm.replace(minute=0)
    tm = tm.strftime("%Y%m%d%H%M")
    stn = "0"
    disp = "0"
    authKey = "ri4L36zbQcuuC9-s27HL3A"
    
    #TA(기온), WD(바람), RN(강수), HM(습도), PS(기압)없으면 각 변수별 정시자료
    # if type == "temp":
    #     var = "TA"
    # elif type == 
    
    return f"https://apihub.kma.go.kr/api/typ01/url/awsh.php?tm={tm}&help=0&authKey={authKey}"

In [9]:
url = return_url_aws()
#url = "https://apihub.kma.go.kr/api/typ01/url/awsh.php?stn=400&authKey=Ns9jp8v2RkSPY6fL9gZEeg"
#url = "https://apihub.kma.go.kr/api/typ01/url/awsh.php?tm=202310080000&help=0&authKey=ri4L36zbQcuuC9-s27HL3A"
response_csv = requests.get(url, verify = False)
if response_csv.status_code == 200:
    csv_data = pd.read_csv(StringIO(response_csv.text))
    print(csv_data.head(10))
        
else:
    print(f"Error: {response_csv.status_code}")
    print(response_csv.text)
    
# pip install pyOpenSSL --user 필요

                                          #START7777
0  # YYMMDDHHMI   STN     TA     WD     WS RN_DAY...
1  #        KST    ID      C    deg    m/s     mm...
2  202310231900    10    5.2  207.4    2.1    0.0...
3  202310231900    12   15.4  176.6    1.7    0.0...
4  202310231900    15  -99.0  -99.0  -99.0  -99.0...
5  202310231900    18   12.8    0.0    0.0    0.0...
6  202310231900    30   12.8  265.5    0.5  -99.0...
7  202310231900    31   16.2  144.5    1.1  -99.0...
8  202310231900    32   12.9  201.8    0.5  -99.0...
9  202310231900    33   11.8  213.9    0.7  -99.0...


/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apihub.kma.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [403]:
column_name = (csv_data.iloc[0, 0].strip().split())[1:]
data_split = [x.strip().split() for x in csv_data.iloc[2:, 0]]
aws_data = pd.DataFrame(data_split, columns = column_name)
aws_data.drop(df.shape[0]-1, axis=0, inplace=True)
aws_data.head(10)

,YYMMDDHHMI,STN,TA,WD,WS,RN_DAY,RN_HR1,HM,PA,PS
0,202310082000,15,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1,202310082000,87,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
3,202310082000,121,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
4,202310082000,211,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
5,202310082000,221,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
6,202310082000,314,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
7,202310082000,322,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
8,202310082000,352,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
9,202310082000,353,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
10,202310082000,360,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


TA: 정시기온 (섭씨)  
WD: 정시 10분 평균 풍향 (degree)  
WS: 정시 10분 평균 풍속 (m/s)  
RN_DAY: 해당시간까지의 일강수량 (mm)  
RN_HR1: 1시간 강수량 (mm)  
HM: 정시습도 (%)  
PA: 정시 현지기압 (hPa)  
PS: 정시 해면기압 (hPa)  

-99.0 maybe missing value.

In [404]:
filtered_aws_data = aws_data.drop([len(aws_data['STN'])], axis = 0)
filtered_aws_data['time'] = pd.to_datetime(aws_data['YYMMDDHHMI'], format = '%Y%m%d%H%M', errors='coerce')
filtered_aws_data = filtered_aws_data.rename(columns={'TA': 'temp', 'WD': 'windDir', 'WS': 'windSpd', 'RN_HR1': 'precip'})
filtered_aws_data.drop(['YYMMDDHHMI', 'RN_DAY', 'HM', 'PA', 'PS'], axis = 1, inplace = True)
filtered_aws_data = filtered_aws_data.astype({'temp' : float, 'windDir' : float, 'windSpd' : float, 'precip': float})
filtered_aws_data

,STN,temp,windDir,windSpd,precip,time
0,15,-99.0,-99.0,-99.0,-99.0,2023-10-08 20:00:00
1,87,-99.0,-99.0,-99.0,-99.0,2023-10-08 20:00:00
3,121,-99.0,-99.0,-99.0,-99.0,2023-10-08 20:00:00
4,211,-99.0,-99.0,-99.0,-99.0,2023-10-08 20:00:00
5,221,-99.0,-99.0,-99.0,-99.0,2023-10-08 20:00:00
6,314,-99.0,-99.0,-99.0,-99.0,2023-10-08 20:00:00
7,322,-99.0,-99.0,-99.0,-99.0,2023-10-08 20:00:00
8,352,-99.0,-99.0,-99.0,-99.0,2023-10-08 20:00:00
9,353,-99.0,-99.0,-99.0,-99.0,2023-10-08 20:00:00
10,360,-99.0,-99.0,-99.0,-99.0,2023-10-08 20:00:00


In [48]:
type(stn_data['지점'].tolist()[0])

int

In [299]:
stn_data = pd.read_csv("OBS_AWS_TIM_20231002162633.csv", encoding = "cp949")
stn_data = stn_data[['지점', '지점명']]
stn_data = stn_data.drop_duplicates()
stn_data = stn_data.reset_index()
stn_data.drop('index', axis = 1, inplace = True)
stn_data['지점'] = stn_data['지점'].astype(str)
stn_info = stn_data['지점'].tolist()
filtered_aws_data = filtered_aws_data[filtered_aws_data['STN'].isin(stn_info)]

In [300]:
stn_dict = stn_data.set_index('지점')['지점명'].to_dict()
filtered_aws_data['NM'] = filtered_aws_data['STN'].map(stn_dict)

/tmp/ipykernel_59416/1479554013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_aws_data['NM'] = filtered_aws_data['STN'].map(stn_dict)


In [301]:
filtered_aws_data

,STN,temp,windDir,windSpd,precip,time,NM
33,116,9.5,326.9,6.7,0.0,2023-10-05 23:00:00,관악(레)
188,400,14.3,288.3,0.9,0.0,2023-10-05 23:00:00,강남
189,401,14.5,242.8,0.8,0.0,2023-10-05 23:00:00,서초
190,402,13.8,25.7,1.3,0.0,2023-10-05 23:00:00,강동
191,403,15.1,324.6,1.3,0.0,2023-10-05 23:00:00,송파
192,404,13.7,317.9,0.9,0.0,2023-10-05 23:00:00,강서
193,405,14.3,357.4,1.4,0.0,2023-10-05 23:00:00,양천
194,406,14.0,309.4,0.7,0.0,2023-10-05 23:00:00,도봉
195,407,9.3,81.2,0.1,0.0,2023-10-05 23:00:00,노원
196,408,14.6,334.5,1.3,0.0,2023-10-05 23:00:00,동대문


## Water Level Data

불러오는 거 6개월까지만 됨...!

In [216]:
#url = "http://www.wamis.go.kr:8080/wamis/openapi/wkw/wl_hrdata"
url = "http://www.wamis.go.kr:8080/wamis/openapi/wkw/wl_dubwlobs"
response_wat_csv = requests.get(url, verify = False)
if response_wat_csv.status_code == 200:
    wat_data = pd.read_csv(StringIO(response_wat_csv.text))
    print(wat_data.head(3))
        
else:
    print(f"Error: {response_wat_csv.status_code}")
    print(response_wat_csv.text)
wat_data = wat_data.columns.tolist()

Empty DataFrame
Columns: [{"result":{"code":"success", msg:"완료되었습니다!"}, count:1007, list:[{"bbsnnm":"한강", obscd:"1001602", obsnm:"평창군(송정교)", clsyn:"운영", obsknd:"T/M", sbsncd:"100108", mngorg:"환경부"}, {"bbsnnm":"한강", obscd:"1001603", obsnm:"태백시(무사교)", clsyn:"운영".1, obsknd:"T/M".1, sbsncd:"100101", mngorg:"한국수자원공사"}, {"bbsnnm":"한강".1, obscd:"1001605", obsnm:"송천1교", clsyn:"운영".2, obsknd:"T/M".2, sbsncd:"100105", mngorg:"한국수력원자력"}, {"bbsnnm":"한강".2, obscd:"1001607", obsnm:"삼척시(번천교)", clsyn:"운영".3, obsknd:"T/M".3, sbsncd:"100101".1, mngorg:"한국수자원공사"}.1, {"bbsnnm":"한강".3, obscd:"1001610", obsnm:"삼척시(광동댐)", clsyn:"폐쇄", obsknd:"T/M".4, sbsncd:"100102", mngorg:"한국수자원공사"}.2, {"bbsnnm":"한강".4, obscd:"1001613", obsnm:"삼척시(광동교)", clsyn:"운영".4, obsknd:"T/M".5, sbsncd:"100102".1, mngorg:"한국수자원공사"}.3, {"bbsnnm":"한강".5, obscd:"1001615", obsnm:"삼척시(갈밭교)", clsyn:"운영".5, obsknd:"T/M".6, sbsncd:"100102".2, mngorg:"한국수자원공사"}.4, {"bbsnnm":"한강".6, obscd:"1001620", obsnm:"정선군(송계교)", clsyn:"운영".6, obsknd:"T/M".7

In [217]:
filtered_wat_data = pd.DataFrame([(wat_data[i], wat_data[i+1]) for i in range(len(wat_data)-1) if i % 7 == 4])
filtered_wat_data = filtered_wat_data.rename(columns={0: 'code', 1: 'name'})
filtered_wat_data['code'] = filtered_wat_data['code'].str.extract(r'obscd:"(\w{7})"')
filtered_wat_data['name'] = filtered_wat_data['name'].str.extract(r'obsnm:"(.*)"')
filtered_wat_data = filtered_wat_data[filtered_wat_data['name'].str.contains('서울시')]

In [218]:
filtered_wat_data

,code,name
213,1018640,서울시(광진교)
216,1018655,서울시(대곡교)
217,1018658,서울시(대치교)
219,1018662,서울시(청담대교)
222,1018669,서울시(창동교)
223,1018670,서울시(월계2교)
224,1018675,서울시(중랑교)
225,1018680,서울시(잠수교)
226,1018683,서울시(한강대교)
229,1018692,서울시(신림5교)


Select the bridges along the 한강: 광진교, 청담대교, 잠수교, 한강대교, 행주대교

In [162]:
filtered_wat_data = filtered_wat_data[(filtered_wat_data['name'] == "서울시(광진교)") | 
                                     (filtered_wat_data['name'] == "서울시(청담대교)") |
                                     (filtered_wat_data['name'] == "서울시(잠수교)") |
                                     (filtered_wat_data['name'] == "서울시(한강대교)") |
                                     (filtered_wat_data['name'] == "서울시(행주대교)")]

In [163]:
filtered_wat_data

,code,name
213,1018640,서울시(광진교)
219,1018662,서울시(청담대교)
225,1018680,서울시(잠수교)
226,1018683,서울시(한강대교)
235,1019630,서울시(행주대교)


1018640: 광진교  
1018662: 청담대교  
1018680: 잠수교  
1018683: 한강대교  
1019630: 행주대교

In [409]:
def make_lvl_dataset(name, startdt, enddt, num_of_seconds_to_wait = 3):
    url = "http://www.wamis.go.kr:8080/wamis/openapi/wkw/wl_hrdata"
    bridge_dict = {'광진교': 1018640, '청담대교': 1018662, '잠수교': 1018680, '한강대교': 1018683}
    code = bridge_dict[name]
    params = {
        'obscd': f'{code}',
        'startdt': f'{startdt}',
        'enddt': f'{enddt}'
    }
    num_of_seconds_to_wait=3
    # if response_wat_kg_csv.status_code == 200:
    #     wat_kg_data = pd.read_csv(StringIO(response_wat_kg_csv.text))
    # else:
    #     print(f"Error: {response_wat_kg_csv.status_code}")
    #     print(response_wat_kg_csv.text)
    # wat_kg_data = wat_kg_data.columns.tolist()
    # return wat_kg_data
    
    try:
        response_wat_csv = requests.get(url, params = params, verify = False)
        
        if response_wat_csv.status_code not in (200, 204, 202):
            if random_num_of_seconds <= 50: # 50초가 넘어가면 멈춘다.
                random_num_of_seconds = random.randint(num_of_seconds_to_wait, num_of_seconds_to_wait + 3)
                sleep(random_num_of_seconds)
                return make_lvl_dataset(name, startdt, enddt,
                                    num_of_seconds_to_wait=num_of_seconds_to_wait + 3)
            else:
                raise Exception(f'Your request failed with the following error: {response_wat_csv.status_code}')
        else:
            wat_data = pd.read_csv(StringIO(response_wat_csv.text))
            wat_data = wat_data.columns.tolist()
            return wat_data
    except Exception as e:
        logging.warning(f'Http request failed with url={url}')
        logging.warning(e)
        raise e

In [383]:
wat_data = make_lvl_dataset('한강대교', 20231007, 20231008)
filtered_wat_data = pd.DataFrame([(wat_data[i], wat_data[i+1]) for i in range(len(wat_data)-1) if i % 2 == 1])
filtered_wat_data.drop([0], axis = 0, inplace = True)
filtered_wat_data = filtered_wat_data.rename(columns={0: 'ymdh', 1: 'lvl'})
filtered_wat_data['ymdh'] = filtered_wat_data['ymdh'].str.extract(r'"ymdh":"(\w{10})"')
filtered_wat_data['lvl'] = filtered_wat_data['lvl'].str.extract(r'wl:"(.*)"')

filtered_wat_data['date'] = filtered_wat_data['ymdh'].str[:-2]
filtered_wat_data['date'] = pd.to_datetime(filtered_wat_data['date'], format='%Y%m%d')
filtered_wat_data['date'].loc[filtered_wat_data['ymdh'].str.endswith('24')] += timedelta(days = 1)
filtered_wat_data['ymdh'].loc[filtered_wat_data['ymdh'].str.endswith('24')] = filtered_wat_data['date'].dt.strftime('%Y%m%d')+"00"

filtered_wat_data['time'] = pd.to_datetime(filtered_wat_data['ymdh'], format = '%Y%m%d%H', errors='coerce')
filtered_wat_data.reset_index(inplace = True)
filtered_wat_data.drop(['index', 'ymdh', 'date'], axis = 1, inplace = True)
filtered_wat_data = filtered_wat_data.astype({'lvl' : float})

/tmp/ipykernel_59416/2481707772.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wat_data['date'].loc[filtered_wat_data['ymdh'].str.endswith('24')] += timedelta(days = 1)
/tmp/ipykernel_59416/2481707772.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wat_data['ymdh'].loc[filtered_wat_data['ymdh'].str.endswith('24')] = filtered_wat_data['date'].dt.strftime('%Y%m%d')+"00"


In [384]:
filtered_wat_data

,lvl,time
0,0.58,2023-10-07 01:00:00
1,0.56,2023-10-07 02:00:00
2,0.55,2023-10-07 03:00:00
3,0.54,2023-10-07 04:00:00
4,0.54,2023-10-07 05:00:00
5,0.52,2023-10-07 06:00:00
6,0.52,2023-10-07 07:00:00
7,0.51,2023-10-07 08:00:00
8,0.50,2023-10-07 09:00:00
9,0.49,2023-10-07 10:00:00


# Matching stations and bridges

Choose three nearest AWS weather stations for each region.  
광진교: 강동, 송파, 광진  
청담대교: 송파, 광진, (강남 or 성동)  
잠수교 & 한강대교: 용산, 영등포, 현충원  
행주대교: 강서 (찾아보니 얘밖에 없는 것 같다 근처에..)

In [167]:
url = "https://apihub.kma.go.kr/api/typ01/url/stn_inf.php?inf=AWS&stn=&tm=202211300900&help=1&authKey=ri4L36zbQcuuC9-s27HL3A"
response_stn_csv = requests.get(url, verify = True)
if response_stn_csv.status_code == 200:
    csv_stn_data = pd.read_csv(StringIO(response_stn_csv.text))
    print(csv_stn_data.head(3))
else:
    print(f"Error: {response_stn_csv.status_code}")
    print(response_stn_csv.text)

                                          #START7777
0  #---------------------------------------------...
1                                    # STN_ID : 지점번호
2           #    LON : 경도(degree) / LAT : 위도(degree)


STN_ID : 지점번호  
LON : 경도(degree) / LAT : 위도(degree)  
STN_SP : 지점 특성코드  
HT : 노장 해발고도 (m)  
HT_PA : 기압계 해발고도 (m)  
HT_TA : 온도계 지상높이 (m)  
HT_WD : 풍향/풍속계 지상높이 (m)  
HT_RN : 우량계 지상높이 (m)  
STN_CD : 지점코드 : 항공등에서 사용  
STN_KO : 지점명(한글)  
STN_EN : 지점명(영문)  
STN_AD : 관리관서번호  
FCT_ID : 예보구역코드  
LAW_ID : 법정동코드  
BASIN : 수계코드  

In [171]:
csv_stn_data = csv_stn_data.iloc[19:]

In [190]:
print(csv_stn_data.iloc[0][0])

   90  128.56473000   38.25085000 41211210     17.53     10.00 4090 105 속초                 ----                 11D20402 5182033035  282


In [209]:
data_stn_split = [x.strip().split() for x in csv_stn_data.iloc[:, 0]]
data_stn_split = data_stn_split[:len(data_stn_split)-1]
data_stn_split = [(sublist[0], sublist[8]) for sublist in data_stn_split]
stn_info = pd.DataFrame(data_stn_split, columns = ["ID", "STN"])

In [212]:
stn_info_ce = stn_info[(stn_info['STN'] == "강동") |
                      (stn_info['STN'] == "광진") |
                      (stn_info['STN'] == "송파") |
                      (stn_info['STN'] == "성동") |
                      (stn_info['STN'] == "강남") |
                      (stn_info['STN'] == "현충원") |
                      (stn_info['STN'] == "영등포") |
                      (stn_info['STN'] == "용산") |
                      (stn_info['STN'] == "강서") |
                      (stn_info['STN'] == "송파")]
print(stn_info_ce)

      ID  STN
172  400   강남
174  402   강동
175  403   송파
176  404   강서
185  413   광진
187  415   용산
192  421   성동
280  510  영등포
639  889  현충원


### some dicts

수위 측정 지점 code : aws 관측소 코드

In [302]:
matching_dict = {1018640 : [402, 403, 413], 1018662: [403, 413, 421], 1018680: [415, 510, 889], 1018683: [415, 510, 990]}

windSpd, windDir 변환 시 각도의 기준을 뭘로 해야 할까?  
한강 기준? 아니면 다리 기준?  
한강 기준이 당연히 맞을 텐데, 경계가 항상 평행한 게 아니라서 측정이 어렵다..  
근사적으로 다리 기준이라고 하기에는 수직이 아닌 경우가 굉장히 많다  
다리 기준으로 하자 귀찮다

다리 각도: 위도선 기준 양의 방향의 각을 측정, 0~pi 범위, theta라 하자  
풍향: phi라 하자.  
ver 성분: -cos(phi+theta)  
hor 성분: sin(phi+theta)

수위 측정 지점 code: 위도선과 다리의 각도

In [385]:
angle_dict = {1018640 : 160, 1018662 : 70, 1018680 : 110, 1018683 : 62}

# Classifying, Funtionization

## aws class

1. excel 파일 있는 만큼을 우선 담아놓는다.  
2. 없는 부분은 계속해서 추가  
3. 파일도 새로 갱신

아 예외 처리 해야 되는데  
파일이 없는 경우: tm1 = 201801010000 으로 설정

In [852]:
colnames_AWS = ["locCode", "name", "time", "temp", "windDir", "windSpd", "precip"]

def climate_data_save(locCode):
    climate_data = pd.DataFrame(columns = colnames_AWS)
    for i in range(2018, 2024):
        path = f"./dataset/OBS_AWS_TIM_{i}_{locCode}.csv"
        if i == 2018:
            climate_data = pd.concat([climate_data, pd.read_csv(path, header = 0, names = colnames_AWS, encoding = 'cp949')], axis = 0)
        else:
            climate_data = pd.concat([climate_data, pd.read_csv(path, header = 0, names = colnames_AWS)], axis = 0)
    climate_data.drop(["locCode", "name"], axis = 1, inplace = True)
    climate_data.to_csv(f"./dataset/climate_data_{locCode}.csv", index = False)

code = [402, 403, 413, 415, 421, 510, 889]
for i in code:
    climate_data_save(i)

In [577]:
df = pd.read_csv("./dataset/climate_data_413.csv", header = 0)
df.head(10)

,time,temp,windDir,windSpd,precip
0,2018-01-01 01:00,-0.8,318.9,1.6,0.0
1,2018-01-01 02:00,-2.0,299.3,0.5,0.0
2,2018-01-01 03:00,-1.1,281.0,1.6,0.0
3,2018-01-01 04:00,-1.6,316.1,0.9,0.0
4,2018-01-01 05:00,-2.7,0.0,0.4,0.0
5,2018-01-01 06:00,-3.3,152.0,0.5,0.0
6,2018-01-01 07:00,-3.9,118.1,0.6,0.0
7,2018-01-01 08:00,-4.4,0.0,0.4,0.0
8,2018-01-01 09:00,-3.3,116.7,1.0,0.0
9,2018-01-01 10:00,-1.5,111.7,0.6,0.0


In [555]:
df['time'].iloc[len(df)-1]

'2023-10-07 23:00'

In [1530]:
class aws2():
    
    code_needed = ["402", "403", "413", "415", "421", "510", "889"]
    colnames_AWS = ["time", "temp", "windDir", "windSpd", "precip"]
    code_dict = {"강동": "402", "광진": "413", "송파": "403", "용산": "415", "성동": "421", "영등포": "510", "현충원": "889"}
    
    def __init__(self):
        
        self.get_past_data()

        tm2 = datetime.now()
        tm1 = self.data_dict[self.code_needed[0]]['time'].iloc[len(self.data_dict[self.code_needed[0]])-1]
        tm1 = datetime.strptime(tm1, '%Y-%m-%d %H:%M')
        tm1 += timedelta(hours = 1)
        
        hourly_times = []
        
        columns = ['STN', 'windDir', 'windSpd', 'precip', 'time']
        self.aws_data = pd.DataFrame(columns = columns)
        #tm1 = datetime(2023, 10, 8, 22, 0)
        cur = tm1
        while cur <= tm2:
            hourly_times.append(cur)
            cur += timedelta(hours=1)

        for time in hourly_times:
            time = time.strftime("%Y%m%d%H%M")
            print("------"+time+"------")
            temp_data = self.make_aws(time)
            self.aws_data = pd.concat([self.aws_data, self.data_preprocessing(temp_data)], axis = 0)
        
        if hourly_times: # Additional data joined.
            self.data_merge()
            self.save_data()
    
    def get_past_data(self):
        self.data_dict = {}
        for code in self.code_needed:
            path = "./dataset/climate_data_"+code+".csv"
            self.data_dict[code] = pd.read_csv(path, header = 0, names = self.colnames_AWS)
    
    def make_aws(self, time, num_of_seconds_to_wait = 3):
        url = f"https://apihub.kma.go.kr/api/typ01/url/awsh.php?tm={time}&help=0&authKey=Ns9jp8v2RkSPY6fL9gZEeg"
        try:
            response_csv = requests.get(url, verify = False)
            if response_csv.status_code not in (200, 204, 202):
                if random_num_of_seconds <= 50:
                    random_num_of_seconds = random.randint(num_of_seconds_to_wait, num_of_seconds_to_wait + 3)
                    sleep(random_num_of_seconds)
                    return make_aws(time, num_of_seconds_to_wait = num_of_seconds_to_wait + 3)
                else:
                    raise Exception(f'Your request failed with the following error: {response_csv.status_code}')
            else:
                aws_data = pd.read_csv(StringIO(response_csv.text))
                return aws_data
        except Exception as e:
            logging.warning(f'Http request failed with url={url}')
            logging.warning(e)
            raise e
            
    def data_preprocessing(self, df):
        df = pd.DataFrame(df)
        column_name = (df.iloc[0, 0].strip().split())[1:]
        data_split = [x.strip().split() for x in df.iloc[2:, 0]]
        aws_data = pd.DataFrame(data_split, columns = column_name)
        aws_data.drop(aws_data.shape[0]-1, axis=0, inplace=True)
        aws_data.reset_index()

        aws_data['time'] = pd.to_datetime(aws_data['YYMMDDHHMI'], format = '%Y%m%d%H%M', errors='coerce')
        aws_data = aws_data.rename(columns={'TA': 'temp', 'WD': 'windDir', 'WS': 'windSpd', 'RN_HR1': 'precip'})
        aws_data = aws_data.astype({'temp' : float, 'windDir' : float, 'windSpd' : float, 'precip': float})
        aws_data = aws_data[aws_data['STN'].isin(self.code_needed)]
        aws_data = aws_data.reset_index()
        aws_data.drop(['index', 'YYMMDDHHMI', 'RN_DAY', 'HM', 'PA', 'PS'], axis = 1, inplace = True)
        aws_data['time'] = aws_data['time'].dt.strftime('%Y-%m-%d %H:%M')
        
        return aws_data
    
    def data_merge(self):
        df = self.aws_data
        for code in self.code_needed:
            df_temp = df[df['STN'] == code]
            df_temp = df_temp.reset_index()
            df_temp.drop(['STN', 'index'], inplace = True, axis = 1)
            df_temp = df_temp[['time', 'temp', 'windDir', 'windSpd', 'precip']]
            self.data_dict[code] = pd.concat([self.data_dict[code], df_temp], axis = 0)
            self.data_dict[code] = self.data_dict[code].reset_index()
            self.data_dict[code].drop(['index'], axis = 1, inplace = True)
            
    def save_data(self):
        for code in self.code_needed:
            self.data_dict[code].to_csv(f"./dataset/climate_data_{code}.csv", index = False)
            
    
            
    def wind_dir_triangulation(self, name, theta):
        theta = math.radians(theta)
        code = self.code_dict[name]
        self.data_dict[code]["vertical_dir"] = -(self.data_dict[code]["windDir"]+theta).map(math.radians).map(math.cos)
        self.data_dict[code]["horizon_dir"] = (self.data_dict[code]["windDir"]+theta).map(math.radians).map(math.sin)
        
    def wind_dir_triangulation(self, code, theta):
        theta = math.radians(theta)
        self.data_dict[code]["vertical_dir"] = -(self.data_dict[code]["windDir"]+theta).map(math.radians).map(math.cos)
        self.data_dict[code]["horizon_dir"] = (self.data_dict[code]["windDir"]+theta).map(math.radians).map(math.sin)
          
    def missingInput(self): #이거 해결해ㅎㅎ 밑에 참고
        for code in self.code_needed:
            print(code)
            i = 0
            while self.data_dict[code].isnull().values.any():
                print(i, end = " ")
                i = i+1
                self.data_dict[code]['precip'] = self.data_dict[code]['precip'].rolling(window = 3, min_periods = 1).mean()
                self.data_dict[code]['horizon_dir'] = self.data_dict[code]['horizon_dir'].rolling(window = 3, min_periods = 1).mean()
                self.data_dict[code]['vertical_dir'] = self.data_dict[code]['vertical_dir'].rolling(window = 3, min_periods = 1).mean()
                self.data_dict[code]['temp'] = self.data_dict[code]['temp'].interpolate().values
                self.data_dict[code]['windSpd'] = self.data_dict[code]['windSpd'].interpolate().values
                
    '''
    여기에 EDA 함수들 적어줘. 풍향 풍속 바꾸는 함수는 만들었고, 위에 결측치 처리 고치고
    '''

In [1531]:
# new version of aws class

class aws:
    
    code_needed = ["402", "403", "413", "415", "421", "510", "889"]
    colnames_AWS = ["time", "temp", "windDir", "windSpd", "precip"]
    code_dict = {"강동": "402", "광진": "413", "송파": "403", "용산": "415", "성동": "421", "영등포": "510", "현충원": "889"}
    columns = ['STN', 'windDir', 'windSpd', 'precip', 'time']
    
    data_dict = dict()
    
    def __init__(self):
        
        self.aws_data = pd.DataFrame(columns = self.columns)
        
    def update_data(self):
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        
        self.get_past_data()
        
        tm2 = datetime.now()
        tm1 = self.data_dict[self.code_needed[0]]['time'].iloc[len(self.data_dict[self.code_needed[0]])-1]
        tm1 = datetime.strptime(tm1, '%Y-%m-%d %H:%M:%S')
        tm1 += timedelta(hours = 1)
        
        hourly_times = []
        
        cur = tm1
        while cur <= tm2:
            hourly_times.append(cur)
            cur += timedelta(hours=1)

        for time in hourly_times:
            time = time.strftime("%Y%m%d%H%M")
            print("------"+time+"------")
            temp_data = self.make_aws(time)
            # print(temp_data[temp_data.iloc[:,1] in ["402", "403", "413", "415", "421", "510", "889"]])
            self.aws_data = pd.concat([self.aws_data, self.data_preprocessing(temp_data)], axis = 0)
        
        if hourly_times: # Additional data has joined
            self.data_merge()
            self.save_data()
    
    @classmethod
    def get_past_data(cls):
        for code in cls.code_needed:
            path = "./dataset/climate_data_"+str(code)+".csv"
            cls.data_dict[code] = pd.read_csv(path, header = 0, names = cls.colnames_AWS)
    
    @staticmethod
    def make_aws(time, num_of_seconds_to_wait = 3):
        url = f"https://apihub.kma.go.kr/api/typ01/url/awsh.php?tm={time}&help=0&authKey=Ns9jp8v2RkSPY6fL9gZEeg"
        try:
            response_csv = requests.get(url, verify = False)
            if response_csv.status_code not in (200, 204, 202):
                if random_num_of_seconds <= 50:
                    random_num_of_seconds = random.randint(num_of_seconds_to_wait, num_of_seconds_to_wait + 3)
                    sleep(random_num_of_seconds)
                    return make_aws(time, num_of_seconds_to_wait = num_of_seconds_to_wait + 3)
                else:
                    raise Exception(f'Your request failed with the following error: {response_csv.status_code}')
            else:
                aws_data = pd.read_csv(StringIO(response_csv.text))
                return aws_data
        except Exception as e:
            logging.warning(f'Http request failed with url={url}')
            logging.warning(e)
            raise e
    
    @staticmethod
    def data_preprocessing(df):
        df = pd.DataFrame(df)
        # code_needed = ["402", "403", "413", "415", "421", "510", "889"]
        
        column_name = (df.iloc[0, 0].strip().split())[1:]
        data_split = [x.strip().split() for x in df.iloc[2:, 0]]
        aws_data = pd.DataFrame(data_split, columns = column_name)
        aws_data.drop(aws_data.shape[0]-1, axis=0, inplace=True)
        aws_data.reset_index()

        aws_data['time'] = pd.to_datetime(aws_data['YYMMDDHHMI'], format = '%Y%m%d%H%M%S', errors='coerce')
        aws_data = aws_data.rename(columns={'TA': 'temp', 'WD': 'windDir', 'WS': 'windSpd', 'RN_HR1': 'precip'})
        aws_data = aws_data.astype({'temp' : float, 'windDir' : float, 'windSpd' : float, 'precip': float})
        aws_data = aws_data[aws_data['STN'].isin(aws.code_needed)]
        aws_data = aws_data.reset_index()
        aws_data.drop(['index', 'YYMMDDHHMI', 'RN_DAY', 'HM', 'PA', 'PS'], axis = 1, inplace = True)
        aws_data['time'] = aws_data['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
        print(aws_data)
        
        return aws_data
    
    def data_merge(self):
        df = self.aws_data
        for code in self.code_needed:
            df_temp = df[df['STN'] == code]
            df_temp = df_temp.reset_index()
            df_temp.drop(['STN', 'index'], inplace = True, axis = 1)
            df_temp = df_temp[['time', 'temp', 'windDir', 'windSpd', 'precip']]
            self.data_dict[code] = pd.concat([self.data_dict[code], df_temp], axis = 0)
            self.data_dict[code] = self.data_dict[code].reset_index()
            self.data_dict[code].drop(['index'], axis = 1, inplace = True)
            
    def save_data(self):
        for code in self.code_needed:
            self.data_dict[code].to_csv(f"./dataset/climate_data_{code}.csv", index = False)
    
    @classmethod
    def save_EDAdata(cls):
        for code in cls.code_needed:
            cls.data_dict[code].to_csv(f"./dataset_cleaning/climate_data_{code}.csv", index = False)
        print("Successfully saved")   
        
    def wind_dir_triangulation(self, name, theta):
        theta = math.radians(theta)
        code = self.code_dict[name]
        self.data_dict[code]["vertical_dir"] = -(self.data_dict[code]["windDir"]+theta).map(math.radians).map(math.cos)
        self.data_dict[code]["horizon_dir"] = (self.data_dict[code]["windDir"]+theta).map(math.radians).map(math.sin)
        
    def wind_dir_triangulation(self, code, theta):
        theta = math.radians(theta)
        self.data_dict[code]["vertical_dir"] = -(self.data_dict[code]["windDir"]+theta).map(math.radians).map(math.cos)
        self.data_dict[code]["horizon_dir"] = (self.data_dict[code]["windDir"]+theta).map(math.radians).map(math.sin)
          
    def missingInput(self):
        for code in self.code_needed:
            print("\n")
            print("code " + str(code) + " proceeding...")
            i = 0
            while self.data_dict[code][['precip', 'horizon_dir', 'vertical_dir', 'temp', 'windSpd']].isnull().values.any():
                print(i, end = "\r")
                i += 1
                self.data_dict[code]['precip'] = self.data_dict[code]['precip'].rolling(window = 3, min_periods = 1).mean()
                self.data_dict[code]['horizon_dir'] = self.data_dict[code]['horizon_dir'].rolling(window = 3, min_periods = 1).mean()
                self.data_dict[code]['vertical_dir'] = self.data_dict[code]['vertical_dir'].rolling(window = 3, min_periods = 1).mean()
                self.data_dict[code]['temp'] = self.data_dict[code]['temp'].interpolate().values
                self.data_dict[code]['windSpd'] = self.data_dict[code]['windSpd'].interpolate().values
                
    @staticmethod         
    def adf_subtest(df, code, col):
        dftest = adfuller(df, autolag="AIC")
        dfoutput = pd.DataFrame({
            "code": code,
            "component": col,
            "Test Statistic": dftest[0],
            "p-value": dftest[1],
            "#Lags Used": dftest[2],
            "Number of Observations Used": dftest[3]
        }, index = [0])

        return dfoutput


    def adf_test(self):
        adf_dict = pd.DataFrame()
        for code in self.code_needed:
            print(str(code) + ": ")
            for i, col in enumerate(self.data_dict[code].columns):
                if i == 0: continue
                print(str(col) + "                    ", end = "\r")
                adf_dict = pd.concat([adf_dict, self.adf_subtest(self.data_dict[code][col], code, col)])
            print("")    
        adf_dict.reset_index()
        return adf_dict
        

In [850]:
A = pd.DataFrame([np.nan, np.nan, 1, 2, 3, 4])
A.rolling(window = 5, min_periods = 1).mean()
A

,0
0,NaN
1,NaN
2,1.0
3,2.0
4,3.0
5,4.0


In [682]:
# code_needed = ["1018640", "1018662", "1018680", "1018683"]

# url = f"http://www.wamis.go.kr:8080/wamis/openapi/wkw/wl_hrdata"
# startdt = 20180101
# enddt = 20180102

# for code in code_needed:
#     params = {
#     'obscd': f'{code}',
#     'startdt': f'{startdt}',
#     'enddt': f'{enddt}'
#     }
#     response_csv = requests.get(url, params = params, verify = False)
#     wat_data = pd.read_csv(StringIO(response_csv.text))
#     wat_data = wat_data.columns.tolist()
#     filtered_wat_data = pd.DataFrame([(wat_data[i], wat_data[i+1]) for i in range(len(wat_data)-1) if i % 2 == 1])
#     filtered_wat_data.drop([0], axis = 0, inplace = True)
#     filtered_wat_data = filtered_wat_data.rename(columns={0: 'ymdh', 1: 'lvl'})
#     filtered_wat_data['ymdh'] = filtered_wat_data['ymdh'].str.extract(r'"ymdh":"(\w{10})"')
#     filtered_wat_data['lvl'] = filtered_wat_data['lvl'].str.extract(r'wl:"(.*)"')

#     filtered_wat_data['date'] = filtered_wat_data['ymdh'].str[:-2]
#     filtered_wat_data['date'] = pd.to_datetime(filtered_wat_data['date'], format='%Y%m%d')
#     filtered_wat_data['date'].loc[filtered_wat_data['ymdh'].str.endswith('24')] += timedelta(days = 1)
#     filtered_wat_data['ymdh'].loc[filtered_wat_data['ymdh'].str.endswith('24')] = filtered_wat_data['date'].dt.strftime('%Y%m%d')+"00"

#     filtered_wat_data['time'] = pd.to_datetime(filtered_wat_data['ymdh'], format = '%Y%m%d%H', errors='coerce')
#     filtered_wat_data.reset_index(inplace = True)
#     filtered_wat_data.drop(['index', 'ymdh', 'date'], axis = 1, inplace = True)
#     filtered_wat_data = filtered_wat_data.astype({'lvl' : float})

#     filtered_wat_data.to_csv(f"./dataset/lvl_data_{code}.csv", index = False)

/tmp/ipykernel_59416/3665603875.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wat_data['date'].loc[filtered_wat_data['ymdh'].str.endswith('24')] += timedelta(days = 1)
/tmp/ipykernel_59416/3665603875.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wat_data['ymdh'].loc[filtered_wat_data['ymdh'].str.endswith('24')] = filtered_wat_data['date'].dt.strftime('%Y%m%d')+"00"
/tmp/ipykernel_59416/3665603875.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

## lvl class

In [1365]:
class lvl2:
    code_needed = ["1018640", "1018662", "1018680", "1018683"]
    angle_dict = {"1018640" : 160, "1018662" : 70, "1018680" : 110, "1018683" : 62}
    matching_dict = {"1018640" : ["402", "403", "413"], "1018662": ["403", "413", "421"], "1018680": ["415", "510", "889"], "1018683": ["415", "510", "889"]}
    colnames_lvl = ["lvl", "time"]
    
    def __init__(self):
        
        self.get_past_data()
        
        tm2 = datetime.now()
        tm2 -= timedelta(days = 1)
        tm2 = tm2.replace(hour = 0, minute = 0, second = 0)
        tm1 = self.data_dict[self.code_needed[0]]['time'].iloc[len(self.data_dict[self.code_needed[0]])-1]
        tm1 = tm1[:10]
        tm1 = datetime.strptime(tm1, '%Y-%m-%d')
        tm1 += timedelta(hours = 1)
        
        hour_list = self.generate_date_ranges(tm1, tm2)
        
    
        for code in code_needed:
            for time_list in hour_list:
                if type(time_list[0]) != str:
                    time_list[0] = time_list[0].strftime("%Y%m%d")
                    time_list[1] = time_list[1].strftime("%Y%m%d")
                    print("------"+time_list[0]+"~"+time_list[1]+"------")
                    #print(self.data_preprocessing(self.make_lvl(code, startdt, enddt)))
                    self.data_dict[code] = pd.concat([self.data_dict[code], self.data_preprocessing(self.make_lvl(code, time_list[0], time_list[1]))], axis = 0)
        
        if tm1 <= tm2: # Additional data joined.
            self.save_data()
    
    def get_past_data(self):
        self.data_dict = {}
        for code in self.code_needed:
            path = "./dataset/lvl_data_"+code+".csv"
            self.data_dict[code] = pd.read_csv(path, header = 0, names = self.colnames_lvl) 
            
            # try: 아니 이거 파일 없으면 except가 실행되어야 하는 거 아니냐고 왜 근데 계속 FileNotFoundError가 뜨는데 아
            #     self.data_dict[code] = pd.read_csv(path, header = 0, names = self.colnames_lvl) 
            # except FileNotFoundError:
            #     pass
            
            
    def generate_date_ranges(self, startdt, enddt, interval_months=6):
        date_ranges = []
        curdt = startdt

        while curdt < enddt:
            nextdt = min(curdt + timedelta(days=interval_months * 30), enddt)
            date_ranges.append([curdt, nextdt])
            curdt = nextdt
        
        return date_ranges
            
    def make_lvl(self, code, startdt, enddt, num_of_seconds_to_wait = 3):
        url = f"http://www.wamis.go.kr:8080/wamis/openapi/wkw/wl_hrdata"
        
        params = {
            'obscd': f'{code}',
            'startdt': f'{startdt}',
            'enddt': f'{enddt}'
        }
        num_of_seconds_to_wait=3
        
        try:
            response_csv = requests.get(url, params = params, verify = False)

            if response_csv.status_code not in (200, 204, 202):
                if random_num_of_seconds <= 50: # 50초가 넘어가면 멈춘다.
                    random_num_of_seconds = random.randint(num_of_seconds_to_wait, num_of_seconds_to_wait + 3)
                    sleep(random_num_of_seconds)
                    return make_lvl(code, startdt, enddt,
                                        num_of_seconds_to_wait=num_of_seconds_to_wait + 3)
                else:
                    raise Exception(f'Your request failed with the following error: {response_csv.status_code}')
            else:
                lvl_data = pd.read_csv(StringIO(response_csv.text))
                lvl_data = lvl_data.columns.tolist()
                return lvl_data
        except Exception as e:
            logging.warning(f'Http request failed with url={url}')
            logging.warning(e)
            raise e
            
            
    def get_past_data(self):
        self.data_dict = {}
        for code in self.code_needed:
            path = "./dataset/lvl_data_"+code+".csv"
            self.data_dict[code] = pd.read_csv(path, header = 0, names = self.colnames_lvl)
            
    def data_preprocessing(self, wat_data):
        filtered_wat_data = pd.DataFrame([(wat_data[i], wat_data[i+1]) for i in range(len(wat_data)-1) if i % 2 == 1])
        filtered_wat_data.drop([0], axis = 0, inplace = True)
        filtered_wat_data = filtered_wat_data.rename(columns={0: 'ymdh', 1: 'lvl'})
        filtered_wat_data['ymdh'] = filtered_wat_data['ymdh'].str.extract(r'"ymdh":"(\w{10})"')
        filtered_wat_data['lvl'] = filtered_wat_data['lvl'].str.extract(r'wl:"(.*)"')

        filtered_wat_data['date'] = filtered_wat_data['ymdh'].str[:-2]
        filtered_wat_data['date'] = pd.to_datetime(filtered_wat_data['date'], format='%Y%m%d')
        filtered_wat_data['date'].loc[filtered_wat_data['ymdh'].str.endswith('24')] += timedelta(days = 1)
        filtered_wat_data['ymdh'].loc[filtered_wat_data['ymdh'].str.endswith('24')] = filtered_wat_data['date'].dt.strftime('%Y%m%d')+"00"

        filtered_wat_data['time'] = pd.to_datetime(filtered_wat_data['ymdh'], format = '%Y%m%d%H', errors='coerce')
        filtered_wat_data.reset_index(inplace = True)
        filtered_wat_data.drop(['index', 'ymdh', 'date'], axis = 1, inplace = True)
        filtered_wat_data = filtered_wat_data.astype({'lvl' : float})
        
        return filtered_wat_data
    
    def save_data(self):
        for code in self.code_needed:
            self.data_dict[code].to_csv(f"./dataset/lvl_data_{code}.csv", index = False)

In [256]:
# new verson of lvl class

class lvl():
    code_needed = ["1018640", "1018662", "1018680", "1018683"]
    angle_dict = {"1018640" : 160, "1018662" : 70, "1018680" : 110, "1018683" : 62}
    matching_dict = {"1018640" : ["402", "403", "413"], "1018662": ["403", "413", "421"], "1018680": ["415", "510", "889"], "1018683": ["415", "510", "889"]}
    colnames_lvl = ["lvl", "time"]
    data_dict = dict()
    
    def __init__(self):
        
        return
        
    def update_data(self):
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

        self.get_past_data()
        
        tm2 = datetime.now()
        tm2 -= timedelta(days = 1)
        tm2 = tm2.replace(hour = 0, minute = 0, second = 0)
        tm1 = self.data_dict[self.code_needed[0]]['time'].iloc[len(self.data_dict[self.code_needed[0]])-1]
        tm1 = tm1[:10]
        tm1 = datetime.strptime(tm1, '%Y-%m-%d')
        tm1 += timedelta(hours = 1)
        
        hour_list = self.generate_date_ranges(tm1, tm2)
    
        for code in self.code_needed:
            print(code+"------------")
            for time_list in hour_list:
                print(time_list)
                if type(time_list[0]) != str:
                    time_list_sub = [0, 0]
                    time_list_sub[0] = time_list[0]
                    time_list_sub[1] = time_list[1]
                    time_list_sub[0] = time_list_sub[0].strftime("%Y%m%d")
                    time_list_sub[1] = time_list_sub[1].strftime("%Y%m%d")
                    print(time_list)
                    print("------"+time_list_sub[0]+"~"+time_list_sub[1]+"------")
                    self.data_dict[code] = pd.concat([self.data_dict[code], self.data_preprocessing(self.make_lvl(code, time_list_sub[0], time_list_sub[1]))], axis = 0).reset_index()[:-1]
        
        
        
        if tm1 <= tm2: # Additional data joined.
            self.save_data()
    
    @classmethod
    def get_past_data(cls):
        for code in cls.code_needed:
            path = "./dataset/lvl_data_"+code+".csv"
            try:
                cls.data_dict[code] = pd.read_csv(path, header = 0, names = cls.colnames_lvl) 
            except FileNotFoundError as e:
                print(f'{e}')
                pass
            # try: 아니 이거 파일 없으면 except가 실행되어야 하는 거 아니냐고 왜 근데 계속 FileNotFoundError가 뜨는데 아
                # self.data_dict[code] = pd.read_csv(path, header = 0, names = self.colnames_lvl) 
            
    @staticmethod        
    def generate_date_ranges(startdt, enddt, interval_months=6):
        date_ranges = []
        curdt = startdt

        while curdt < enddt:
            nextdt = min(curdt + timedelta(days=interval_months * 30), enddt)
            date_ranges.append([curdt, nextdt])
            curdt = nextdt
        
        return date_ranges
    
    @staticmethod
    def make_lvl(code, startdt, enddt, num_of_seconds_to_wait = 3):
        url = f"http://www.wamis.go.kr:8080/wamis/openapi/wkw/wl_hrdata"
        
        params = {
            'obscd': f'{code}',
            'startdt': f'{startdt}',
            'enddt': f'{enddt}'
        }
        num_of_seconds_to_wait=3
        
        try:
            response_csv = requests.get(url, params = params, verify = False)

            if response_csv.status_code not in (200, 204, 202):
                if random_num_of_seconds <= 50: # 50초가 넘어가면 멈춘다.
                    random_num_of_seconds = random.randint(num_of_seconds_to_wait, num_of_seconds_to_wait + 3)
                    sleep(random_num_of_seconds)
                    return make_lvl(code, startdt, enddt,
                                        num_of_seconds_to_wait=num_of_seconds_to_wait + 3)
                else:
                    raise Exception(f'Your request failed with the following error: {response_csv.status_code}')
            else:
                lvl_data = pd.read_csv(StringIO(response_csv.text))
                lvl_data = lvl_data.columns.tolist()
                return lvl_data
        except Exception as e:
            logging.warning(f'Http request failed with url={url}')
            logging.warning(e)
            raise e
        
       
    @staticmethod
    def data_preprocessing(wat_data):
        filtered_wat_data = pd.DataFrame([(wat_data[i], wat_data[i+1]) for i in range(len(wat_data)-1) if i % 2 == 1])
        filtered_wat_data.drop([0], axis = 0, inplace = True)
        filtered_wat_data = filtered_wat_data.rename(columns={0: 'ymdh', 1: 'lvl'})
        filtered_wat_data['ymdh'] = filtered_wat_data['ymdh'].str.extract(r'"ymdh":"(\w{10})"')
        filtered_wat_data['lvl'] = filtered_wat_data['lvl'].str.extract(r'wl:"(.*)"')

        filtered_wat_data['date'] = filtered_wat_data['ymdh'].str[:-2]
        filtered_wat_data['date'] = pd.to_datetime(filtered_wat_data['date'], format='%Y%m%d')
        filtered_wat_data['date'].loc[filtered_wat_data['ymdh'].str.endswith('24')] += timedelta(days = 1)
        filtered_wat_data['ymdh'].loc[filtered_wat_data['ymdh'].str.endswith('24')] = filtered_wat_data['date'].dt.strftime('%Y%m%d')+"00:00:00"

        filtered_wat_data['time'] = pd.to_datetime(filtered_wat_data['ymdh'], format = '%Y%m%d%H', errors='coerce')
        filtered_wat_data.reset_index(inplace = True)
        filtered_wat_data.drop(['index', 'ymdh', 'date'], axis = 1, inplace = True)
        filtered_wat_data = filtered_wat_data.astype({'lvl' : float})
        
        
        return filtered_wat_data
    
    def save_data(self):
        for code in self.code_needed:
            self.data_dict[code].to_csv(f"./dataset/lvl_data_{code}.csv", index = False)
            
    @classmethod
    def save_EDAdata(cls):
        for code in cls.code_needed:
            cls.data_dict[code].to_csv(f"./dataset_cleaning/lvl_data_{code}.csv", index = False)
        print("Successfully saved")
            
    @staticmethod         
    def adf_subtest(df, code, col):
        dftest = adfuller(df, autolag="AIC")
        dfoutput = pd.DataFrame({
            "code": code,
            "component": col,
            "Test Statistic": dftest[0],
            "p-value": dftest[1],
            "#Lags Used": dftest[2],
            "Number of Observations Used": dftest[3]
        }, index = [0])

        return dfoutput


    def adf_test(self):
        adf_dict = pd.DataFrame()
        for code in self.code_needed:
            print(str(code) + ": waterlevel")
            adf_dict = pd.concat([adf_dict, self.adf_subtest(self.data_dict[code]["lvl"], code, "lvl")])
            print("")    
        adf_dict.reset_index()
        return adf_dict
    
    def missingInput(self):
        for code in self.code_needed:
            print("\n")
            print("code " + str(code) + " proceeding...")
            i = 0
            while self.data_dict[code][['lvl']].isnull().values.any():
                print(i, end = "\r")
                i += 1
                self.data_dict[code]['lvl'] = self.data_dict[code]['lvl'].interpolate().values

In [257]:
lvl_1018640.to_csv("./dataset/lvl_data_1018640.csv", index = False)
lvl_1018680.to_csv("./dataset/lvl_data_1018680.csv", index = False)
lvl_1018662.to_csv("./dataset/lvl_data_1018662.csv", index = False)
lvl_1018683.to_csv("./dataset/lvl_data_1018683.csv", index = False)

In [258]:
LVL = lvl()

In [260]:
LVL.update_data()

1018640------------
[datetime.datetime(2023, 10, 9, 1, 0), datetime.datetime(2023, 10, 13, 0, 0, 0, 558881)]
[datetime.datetime(2023, 10, 9, 1, 0), datetime.datetime(2023, 10, 13, 0, 0, 0, 558881)]
------20231009~20231013------
1018662------------
[datetime.datetime(2023, 10, 9, 1, 0), datetime.datetime(2023, 10, 13, 0, 0, 0, 558881)]
[datetime.datetime(2023, 10, 9, 1, 0), datetime.datetime(2023, 10, 13, 0, 0, 0, 558881)]
------20231009~20231013------


/tmp/ipykernel_17709/3691945890.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wat_data['date'].loc[filtered_wat_data['ymdh'].str.endswith('24')] += timedelta(days = 1)
/tmp/ipykernel_17709/3691945890.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wat_data['ymdh'].loc[filtered_wat_data['ymdh'].str.endswith('24')] = filtered_wat_data['date'].dt.strftime('%Y%m%d')+"00:00:00"
/tmp/ipykernel_17709/3691945890.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

1018680------------
[datetime.datetime(2023, 10, 9, 1, 0), datetime.datetime(2023, 10, 13, 0, 0, 0, 558881)]
[datetime.datetime(2023, 10, 9, 1, 0), datetime.datetime(2023, 10, 13, 0, 0, 0, 558881)]
------20231009~20231013------
1018683------------
[datetime.datetime(2023, 10, 9, 1, 0), datetime.datetime(2023, 10, 13, 0, 0, 0, 558881)]
[datetime.datetime(2023, 10, 9, 1, 0), datetime.datetime(2023, 10, 13, 0, 0, 0, 558881)]
------20231009~20231013------


/tmp/ipykernel_17709/3691945890.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wat_data['date'].loc[filtered_wat_data['ymdh'].str.endswith('24')] += timedelta(days = 1)
/tmp/ipykernel_17709/3691945890.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wat_data['ymdh'].loc[filtered_wat_data['ymdh'].str.endswith('24')] = filtered_wat_data['date'].dt.strftime('%Y%m%d')+"00:00:00"
/tmp/ipykernel_17709/3691945890.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [261]:
LVL.data_dict

{'1018640':        index   lvl                 time
 0          0  1.06  2018-01-01 01:00:00
 1          1  1.06  2018-01-01 02:00:00
 2          2  1.06  2018-01-01 03:00:00
 3          3  1.05  2018-01-01 04:00:00
 4          4  1.05  2018-01-01 05:00:00
 ...      ...   ...                  ...
 50705    114  1.13  2023-10-13 19:00:00
 50706    115  1.13  2023-10-13 20:00:00
 50707    116  1.13  2023-10-13 21:00:00
 50708    117  1.12  2023-10-13 22:00:00
 50709    118  1.12  2023-10-13 23:00:00
 
 [50710 rows x 3 columns],
 '1018662':        index   lvl                 time
 0          0  1.20  2018-01-01 01:00:00
 1          1  1.19  2018-01-01 02:00:00
 2          2  1.15  2018-01-01 03:00:00
 3          3  1.13  2018-01-01 04:00:00
 4          4  1.12  2018-01-01 05:00:00
 ...      ...   ...                  ...
 50705    114  1.08  2023-10-13 19:00:00
 50706    115  1.42  2023-10-13 20:00:00
 50707    116  1.68  2023-10-13 21:00:00
 50708    117  1.32  2023-10-13 22:00:00
 50709

In [263]:
LVL.data_dict["1018640"].isnull().sum()

index    0
lvl      0
time     4
dtype: int64

In [262]:
LVL.missingInput()



code 1018640 proceeding...
0

code 1018662 proceeding...
0

code 1018680 proceeding...
0

code 1018683 proceeding...


In [264]:
LVL.save_EDAdata()

Successfully saved


## data preprocessing

In [1533]:
AWS = aws()
LVL = lvl()

In [1526]:
AWS.get_past_data()

In [1527]:
AWS.data_dict

{'402':                       time  temp  windDir  windSpd  precip
 0      2018-01-01 01:00:00  -1.8    349.5      0.7     0.0
 1      2018-01-01 02:00:00  -2.7      9.4      0.6     0.0
 2      2018-01-01 03:00:00  -3.2     82.7      0.7     0.0
 3      2018-01-01 04:00:00  -4.1      0.0      0.4     0.0
 4      2018-01-01 05:00:00  -4.9     95.2      1.7     0.0
 ...                    ...   ...      ...      ...     ...
 50587  2023-10-09 20:00:00  17.9    251.6      0.7     0.0
 50588  2023-10-09 21:00:00  17.0     10.0      0.5     0.0
 50589  2023-10-09 22:00:00  15.7    328.3      1.7     0.0
 50590  2023-10-09 23:00:00  14.7    328.4      1.1     0.0
 50591  2023-10-10 00:00:00  14.0     11.4      1.2     0.0
 
 [50592 rows x 5 columns],
 '403':                       time  temp  windDir  windSpd  precip
 0      2018-01-01 01:00:00  -0.5    308.9      2.3     0.0
 1      2018-01-01 02:00:00  -0.7    300.2      1.3     0.0
 2      2018-01-01 03:00:00  -0.6    310.3      1.5     0

In [1534]:
AWS.update_data()

------202310100100------
   STN  temp  windDir  windSpd  precip                 time
0  402  13.9     10.4      1.0     0.0  2023-10-10 01:00:00
1  403  15.5    313.2      1.4     0.0  2023-10-10 01:00:00
2  413  15.3    313.4      0.9     0.0  2023-10-10 01:00:00
3  415  15.0    214.3      0.8     0.0  2023-10-10 01:00:00
4  421  15.0    279.1      2.2     0.0  2023-10-10 01:00:00
5  510  14.9     21.0      0.6     0.0  2023-10-10 01:00:00
6  889  12.4    235.8      0.5     0.0  2023-10-10 01:00:00
------202310100200------
   STN  temp  windDir  windSpd  precip                 time
0  402  13.3     85.5      1.5     0.0  2023-10-10 02:00:00
1  403  15.2    334.3      0.6     0.0  2023-10-10 02:00:00
2  413  14.1     59.1      0.3     0.0  2023-10-10 02:00:00
3  415  14.5    338.4      0.3     0.0  2023-10-10 02:00:00
4  421  14.6     20.5      1.4     0.0  2023-10-10 02:00:00
5  510  14.6    291.2      1.0     0.0  2023-10-10 02:00:00
6  889  12.1    229.8      0.5     0.0  2023-10-10

In [1535]:
AWS.update_data()

In [92]:
LVL = lvl()

In [93]:
LVL.update_data()

------20231009~20231013------


/tmp/ipykernel_17709/3792811481.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wat_data['date'].loc[filtered_wat_data['ymdh'].str.endswith('24')] += timedelta(days = 1)
/tmp/ipykernel_17709/3792811481.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wat_data['ymdh'].loc[filtered_wat_data['ymdh'].str.endswith('24')] = filtered_wat_data['date'].dt.strftime('%Y%m%d')+"00:00:00"


In [94]:
LVL.data_dict

{'1018640': index     119
 lvl      1.11
 time      NaT
 Name: 50710, dtype: object,
 '1018662':         lvl                 time
 0      1.20  2018-01-01 01:00:00
 1      1.19  2018-01-01 02:00:00
 2      1.15  2018-01-01 03:00:00
 3      1.13  2018-01-01 04:00:00
 4      1.12  2018-01-01 05:00:00
 ...     ...                  ...
 50586  1.18  2023-10-09 19:00:00
 50587  1.19  2023-10-09 20:00:00
 50588  1.23  2023-10-09 21:00:00
 50589  1.23  2023-10-09 22:00:00
 50590  1.23  2023-10-09 23:00:00
 
 [50591 rows x 2 columns],
 '1018680':         lvl                 time
 0      2.92  2018-01-01 01:00:00
 1      2.90  2018-01-01 02:00:00
 2      2.86  2018-01-01 03:00:00
 3      2.85  2018-01-01 04:00:00
 4      2.83  2018-01-01 05:00:00
 ...     ...                  ...
 50586  2.88  2023-10-09 19:00:00
 50587  2.90  2023-10-09 20:00:00
 50588  2.93  2023-10-09 21:00:00
 50589  2.94  2023-10-09 22:00:00
 50590  2.94  2023-10-09 23:00:00
 
 [50591 rows x 2 columns],
 '1018683':        

In [1614]:
for lvl_code in LVL.code_needed:
    for aws_code in LVL.matching_dict[lvl_code]:
        AWS.wind_dir_triangulation(aws_code, LVL.angle_dict[lvl_code])

In [39]:
AWS.data_dict["403"].isnull().sum()

NameError: name 'AWS' is not defined

In [1617]:
AWS.missingInput()



code 402 proceeding...
435

code 403 proceeding...
3384

code 413 proceeding...
27

code 415 proceeding...
41

code 421 proceeding...
470

code 510 proceeding...
41

code 889 proceeding...


AttributeError: 'lvl' object has no attribute 'missingInput'

아니 왤케 느려 미친 거 아니야?
ㅁ이러ㅏㄴ모;ㅑㅐㅈ디ㅓㄹ아ㅚㅕㅐㅁㅈ댜ㅣㅓㅇ나로ㅕㅣ재ㅑ더밀ㅇㄴ파
저 거위도 벽을 너머 하늘을 날을 거라고
달팽이도 넓고 거친 바다 끝에 꿈을 둔다고
나도 꾸물꾸물 말고 꿈을 찾을래
어서 남의 꿈을 빌려 꾸기라도 해
내게 강요하지 말아요
이건 내 길이 아닌 걸
내밀지 말아요 너의 구겨진 꿈을
난 차라리 흘러갈래
모두 높은 곳을 우러러볼 때
난 내 물결을 따라
flow flow along flow along my way
난 차라리 꽉 눌러붙을래
날 재촉한다면
따뜻한 밥 위에 누워자는
계란 fry fry 같이 나른하게
fry fry fry fry
고래도 사랑을 찾아 파도를 가를 거라고
하다못해 네모도 꿈을 꾸는데
아무도 꿈이 없는 자엑에 기회를 주지 않아
하긴 무슨 기회가 어울릴지도 모를 거야
무시 말아줘요 하고 싶은 게 없는 걸
왜 그렇게 봐 난 죄지은 게 아닌데
난 차라리 흘러갈래
모두 높은 곳을 우러러볼 때
난 내 물결을 따라
flow flow along flow along my way
난 차라리 꽉 눌러붙을래


In [986]:
# 이것은 노래 가사로 보입니다. 노래는 주로 감정이나 경험을 표현하고 공유하는 매체로 사용됩니다. 
# 이 가사에서는 누군가가 자신의 삶이나 꿈을 향한 욕망에 대한 감정을 표현하고 있습니다. 
# 가사의 내용을 간단히 설명하면, 누군가가 자신의 삶이 너무 느리고 변화가 없다고 느끼며, 
# 다른 사람들의 꿈을 참조하거나 영감을 얻어 새로운 방향을 찾고자 하는 것 같습니다. 
# 그러나 그들은 다른 사람들의 꿈이나 기회를 복제하려고 하는 것이 아니라, 자신만의 방식으로 나아가려고 합니다. 
# 마지막으로, 그들은 자신의 길을 찾기 위해 힘들게 노력하고 있음을 나타내고 있습니다. 
# 이 가사는 어떤 상황이나 감정을 표현하고자 하는 것으로 보이며, 노래의 음악적인 부분과 함께 들을 때 
# 더욱 감정적인 의미를 갖을 것입니다.

In [1620]:
for aws_code in AWS.code_needed:
    AWS.data_dict[aws_code]['vertical_dir'] = AWS.data_dict[aws_code]['vertical_dir'] * AWS.data_dict[aws_code]['windSpd']
    AWS.data_dict[aws_code]['horizon_dir'] = AWS.data_dict[aws_code]['horizon_dir'] * AWS.data_dict[aws_code]['windSpd']
    AWS.data_dict[aws_code].drop(['windDir', 'windSpd'], axis = 1, inplace = True)

In [63]:
LVL.data_dict

{'1018640': index     119
 lvl      1.11
 time      NaT
 Name: 50710, dtype: object,
 '1018662':         lvl                 time
 0      1.20  2018-01-01 01:00:00
 1      1.19  2018-01-01 02:00:00
 2      1.15  2018-01-01 03:00:00
 3      1.13  2018-01-01 04:00:00
 4      1.12  2018-01-01 05:00:00
 ...     ...                  ...
 50586  1.18  2023-10-09 19:00:00
 50587  1.19  2023-10-09 20:00:00
 50588  1.23  2023-10-09 21:00:00
 50589  1.23  2023-10-09 22:00:00
 50590  1.23  2023-10-09 23:00:00
 
 [50591 rows x 2 columns],
 '1018680':         lvl                 time
 0      2.92  2018-01-01 01:00:00
 1      2.90  2018-01-01 02:00:00
 2      2.86  2018-01-01 03:00:00
 3      2.85  2018-01-01 04:00:00
 4      2.83  2018-01-01 05:00:00
 ...     ...                  ...
 50586  2.88  2023-10-09 19:00:00
 50587  2.90  2023-10-09 20:00:00
 50588  2.93  2023-10-09 21:00:00
 50589  2.94  2023-10-09 22:00:00
 50590  2.94  2023-10-09 23:00:00
 
 [50591 rows x 2 columns],
 '1018683':        

In [41]:
LVL.data_dict["1018640"].isnull().sum()

lvl     3
time    4
dtype: int64

In [1013]:
AWS.data_dict["889"].isnull().sum()

time            0
temp            0
precip          0
vertical_dir    0
horizon_dir     0
dtype: int64

In [1022]:
AWS.adf_test()

402: 
horizon_dir                     
403: 
horizon_dir                     
413: 
horizon_dir                     
415: 
horizon_dir                     
421: 
horizon_dir                     
510: 
horizon_dir                     
889: 
horizon_dir                     


,code,component,Test Statistic,p-value,#Lags Used,Number of Observations Used
0,402,temp,-4.968714,2.558462e-05,48,24288
0,402,precip,-33.905837,0.000000e+00,15,24321
0,402,vertical_dir,-19.342946,0.000000e+00,45,24291
0,402,horizon_dir,-16.073800,5.470751e-29,48,24288
0,403,temp,-5.079102,1.544080e-05,48,24252
0,403,precip,-35.136805,0.000000e+00,15,24285
0,403,vertical_dir,-17.163053,6.821578e-30,39,24261
0,403,horizon_dir,-20.423060,0.000000e+00,24,24276
0,413,temp,-4.939138,2.924637e-05,48,24243
0,413,precip,-11.413484,7.155494e-21,18,24273


In [1052]:
LVL.adf_test()

1018640: waterlevel

1018662: waterlevel

1018680: waterlevel

1018683: waterlevel



,code,component,Test Statistic,p-value,#Lags Used,Number of Observations Used
0,1018640,lvl,-10.926613,1.006456e-19,57,51348
0,1018662,lvl,-3.114115,2.552610e-02,1,46
0,1018680,lvl,-2.457722,1.261026e-01,3,44
0,1018683,lvl,-1.521151,5.230234e-01,2,45


MSTL을 class method로 포함시키기에는 시간이 너무 오래 걸려서 리스크가 클 것 같다. 따라서 MSTL은 단독적인 하나의 python 파일로 만들고 그 전까지의 EDA process로 가공된 데이터를 `data_cleaning` 폴더에 저장한다. 가공된 데이터를 저장하는 method 함수는 save_EDAdata method로 제작하였다.

In [1623]:
AWS.save_EDAdata()

Successfully saved


In [15]:
LVL.save_EDAdata()

Successfully saved


이 파일의 끝.

**++ MSTL is made in a new object.**

In [448]:
def data_preprocessing_aws(file_path):
    # open file
    file_path = file_path
    
    with open(file_path, 'r', encoding='cp949') as file:
        lines = file.readlines()
    
    # transform txt to dataframe
    column_name = (lines[20].strip().split())[1:]
    data_split = [x.strip().split() for x in lines[22:]]
    df = pd.DataFrame(data_split, columns = column_name)
    df.drop(df.shape[0]-1, axis=0, inplace=True)
    
    # retain only the data corresponding to 00 minutes and remove the rest
    df = df[df['YYMMDDHHMI'].str.endswith('00')]
    
    return df

In [244]:
def download_wat():
    obscd = 0
    startdt = 0
    enddt = 0
    url = f"http://www.wamis.go.kr:8080/wamis/openapi/wkw/wl_hrdata?obscd={obscd}&startdt={startdt}&enddt={enddt}&output=xml&key={key}"
    save_file_path = "wat_file_temp.txt"
    download_file(url, save_file_path)
    return save_file_path

In [ ]:
def data_preprocessing_wat(file_path):
    
    file_path = file_path
    
    with open(file_path, 'r', encoding='cp949') as file:
        lines = file.readlines()
    
    df = pd.DataFrame(data_split, columns = column_name)
    

In [250]:
url = "https://apihub.kma.go.kr/api/typ01/url/awsh.php?var=TA&tm=201508121500&help=1&authKey=ri4L36zbQcuuC9-s27HL3A"
save_file_path = "aws_files_temp_hour.txt"
download_file(url, save_file_path)
with open(save_file_path, 'r') as file:
    lines = file.readlines()

In [251]:
lines

['{\n',
 '  "result" : {\n',
 '    "status" : 403,\n',
 '    "message" : "활용신청이 필요한 API 입니다. 활용신청 후 다시 시도해 주십시오."\n',
 '  }\n',
 '}']

In [407]:
now = datetime.now()
now = now.replace(minute = 0)
now = now.strftime("%Y%m%d%H%M")
tm2 = now
tm1 = str(int(now) - 500000000)
print(tm1)

201810082000


In [256]:
now = datetime.now()
now = now.strftime("%Y%m%d%H%M")
tm2 = now
tm1 = str(int(now) - 500000000)
print(tm1)
stn = "0"
disp = "0"
help = "1"
authKey = "ri4L36zbQcuuC9-s27HL3A"
url =  f"https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?tm1={tm1}&tm2={tm2}&stn={stn}&disp={disp}&help={help}&authKey={authKey}"
x = requests.get(url, verify = False)
print(x)

201810051443


/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apihub.kma.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>


In [246]:
with open(file_path, 'r') as file:
    lines = file.readlines()

In [247]:
lines[0]

'{"result":{"code":"failure","msg":"필수 요청 항목이 없습니다!"}}'

In [255]:
file_path = download_aws()
df_aws = data_preprocessing_aws(file_path)

201810051436


ConnectionError: HTTPSConnectionPool(host='apihub.kma.go.kr', port=443): Max retries exceeded with url: /api/typ01/cgi-bin/url/nph-aws2_min?tm1=201810051436&tm2=202310051436&stn=0&disp=0&help=1&authKey=ri4L36zbQcuuC9-s27HL3A (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd5c19e52e0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [ ]:
LVL = lvl()

------------

In [221]:
df.head(5)

,YYMMDDHHMI,STN,WD1,WS1,WDS,WSS,WS10,WS10,TA,RE,RN-15m,RN-60m,RN-12H,RN-DAY,HM,PA,PS,TD
35574,202310050900,90,355.7,1.9,349.7,3.1,304.8,1.5,15.4,0.0,0.0,0.0,0.0,0.0,29.9,1013.5,1015.6,-2.1
35575,202310050900,92,270.0,2.8,300.0,4.4,280.0,3.1,15.0,-99.9,0.0,0.0,0.0,0.0,38.0,1006.7,1015.7,0.8
35576,202310050900,93,167.4,1.3,168.8,1.6,173.4,1.0,11.5,0.0,0.0,0.0,0.0,0.0,70.4,1006.2,1017.7,6.3
35577,202310050900,95,137.8,0.9,137.8,1.4,162.3,0.9,11.1,0.0,0.0,0.0,0.0,0.0,54.9,999.6,1018.3,2.4
35578,202310050900,96,120.6,1.9,207.0,4.2,87.9,4.2,17.7,0.0,0.0,0.0,0.0,0.0,52.5,999.4,1010.9,7.9


In [213]:
df.tail(5)

,YYMMDDHHMI,STN,WD1,WS1,WDS,WSS,WS10,WS10,TA,RE,RN-15m,RN-60m,RN-12H,RN-DAY,HM,PA,PS,TD
107443,202310042100,989,8.1,3.6,25.3,5.2,3.6,3.7,18.5,0.0,0.0,0.0,0.0,0.0,86.5,1003.4,1013.9,16.2
107444,202310042100,990,39.6,4.2,23.1,5.8,42.3,4.3,19.9,0.0,0.0,0.0,0.0,0.0,76.5,1005.4,1014.5,15.7
107445,202310042100,991,217.0,2.0,237.9,2.3,233.0,1.2,17.5,0.0,0.0,0.0,0.0,0.0,87.5,1009.1,1012.3,15.4
107446,202310042100,992,227.5,0.8,230.6,1.2,241.2,0.8,17.0,0.0,0.0,0.0,0.0,0.0,89.4,1007.8,1012.0,15.2
107447,202310042100,994,252.5,5.7,250.3,5.9,253.8,5.0,20.8,0.0,0.0,0.0,0.0,0.0,79.6,1013.8,1014.2,17.1


In [52]:
file_path = 'api_files.txt'

with open(file_path, 'r', encoding='cp949') as file:
    lines = file.readlines()

# 각 줄 출력 또는 다른 작업 수행
for line in lines:
    print(line)
    
 # '#  WD1    : 1분 평균 풍향 (degree) : 0-N, 90-E, 180-S, 270-W, 360-무풍 \n',
 # '#  WS1    : 1분 평균 풍속 (m/s) \n',
 # '#  WDS    : 최대 순간 풍향 (degree) \n',
 # '#  WSS    : 최대 순간 풍속 (m/s) \n',
 # '#  WD10   : 10분 평균 풍향 (degree) \n',
 # '#  WS10   : 10분 평균 풍속 (m/s) \n',
 # '#  TA     : 1분 평균 기온 (C) \n',
 # '#  RE     : 강수감지 (0-무강수, 0이 아니면-강수) \n',
 # '#  RN-15m : 15분 누적 강수량 (mm) \n',
 # '#  RN-60m : 60분 누적 강수량 (mm) \n',
 # '#  RN-12H : 12시간 누적 강수량 (mm) \n',
 # '#  RN-DAY : 일 누적 강수량 (mm) \n',
 # '#  HM     : 1분 평균 상대습도 (%) \n',
 # '#  PA     : 1분 평균 현지기압 (hPa) \n',
 # '#  PS     : 1분 평균 해면기압 (hPa) \n',
 # '#  TD     : 이슬점온도 (C) \n',
 # '#  *) -50 이하면 관측이 없거나, 에러처리된 것을 표시 \n',

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [114]:
df.shape[0]-1

131224

In [179]:
column_name = (lines[20].strip().split())[1:]
data_split = [x.strip().split() for x in lines[22:]]
df = pd.DataFrame(data_split, columns = column_name)
df.drop(df.shape[0]-1, axis=0, inplace=True)

In [77]:
df.head(180)

,YYMMDDHHMI,STN,WD1,WS1,WDS,WSS,WS10,WS10,TA,RE,RN-15m,RN-60m,RN-12H,RN-DAY,HM,PA,PS,TD
0,202302010600,90,312.7,1.9,329.9,2.9,308.2,2.2,4.1,0.0,0.0,0.0,0.0,0.0,41.2,1011.5,1013.7,-7.9
1,202302010600,92,280.0,3.5,290.0,4.4,270.0,4.6,4.1,-99.9,0.0,0.0,0.0,0.0,47.0,1004.3,1013.6,-6.2
2,202302010600,93,0.0,0.0,121.0,0.0,152.0,0.5,-6.0,0.0,0.0,0.0,0.0,0.0,95.6,1004.1,1016.4,-6.6
3,202302010600,95,88.3,0.1,98.5,0.2,41.1,0.2,-3.7,0.0,0.0,0.0,0.0,0.0,97.3,997.3,1017.1,-4.1
4,202302010600,96,261.6,15.7,263.6,18.1,262.3,15.8,8.9,-99.5,0.0,0.0,-99.9,0.0,52.7,999.4,1011.4,-0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,202302010600,402,27.1,0.9,13.5,1.2,54.9,0.4,0.8,0.0,0.0,0.0,0.0,0.0,100.0,1009.0,1015.9,0.8
176,202302010600,403,304.0,1.3,348.8,1.5,311.5,1.6,2.0,0.0,0.0,0.0,0.0,0.0,-99.7,-99.7,-99.7,-99.9
177,202302010600,404,337.3,1.5,348.8,2.2,317.8,1.6,0.5,0.0,0.0,0.0,0.0,0.0,-99.7,-99.7,-99.7,-99.9
178,202302010600,405,271.2,0.7,284.4,1.0,281.6,1.2,1.0,0.0,0.0,0.0,0.0,0.0,96.6,1013.8,1015.3,0.5


# Convert minute-unit data to hour-unit data

정시 기온은 해당 시간의 00분 기온을 의미  
정시 풍향․풍속은 해당 시각의 10분 전부터 정시까지의 10개의 1분 평균 풍향․풍속의 평균값으로 구한다.  
1시간 최다강수량은 1분 강수량을 이용하여 전일 23:31～익일 00:30까지 1시간 이동 누적값을 계산하고 이 이동 누적값 중 최다값과 그 때의 시각을 구한다.

WD1    : 1분 평균 풍향 (degree) : 0-N, 90-E, 180-S, 270-W, 360-무풍  
WS1    : 1분 평균 풍속 (m/s)  
WD10   : 10분 평균 풍향 (degree)  
WS10   : 10분 평균 풍속 (m/s)  
TA     : 1분 평균 기온 (C)  
RN-60m : 60분 누적 강수량 (mm)  
TD     : 이슬점온도 (C)

In [181]:
df['YY'] = df['YYMMDDHHMI'].str[:4]
df['MM'] = df['YYMMDDHHMI'].str[4:6]
df['DD'] = df['YYMMDDHHMI'].str[6:8]
df['HH'] = df['YYMMDDHHMI'].str[8:10]
df['MI'] = df['YYMMDDHHMI'].str[10:12]

In [186]:
df_temp['YYMMDDHHMI'].unique()

array(['202302010600', '202302010700', '202302010800', '202302010900'],
      dtype=object)

In [185]:
df.shape

(131225, 23)

In [187]:
df.iloc[df.shape[0]-1]

YYMMDDHHMI    202302010900
STN                    994
WD1                  294.0
WS1                    3.0
WDS                  292.5
WSS                    3.2
WS10                 301.7
WS10                   3.2
TA                     3.7
RE                   -99.5
RN-15m                 0.0
RN-60m                 0.0
RN-12H                 0.0
RN-DAY                 0.0
HM                    96.2
PA                  1017.9
PS                  1018.3
TD                     3.2
YY                    2023
MM                      02
DD                      01
HH                      09
MI                      00
Name: 131224, dtype: object

In [183]:
df_temp = df[df['MI'] == '00']
df_temp.iloc[2899]

YYMMDDHHMI    202302010900
STN                    994
WD1                  294.0
WS1                    3.0
WDS                  292.5
WSS                    3.2
WS10                 301.7
WS10                   3.2
TA                     3.7
RE                   -99.5
RN-15m                 0.0
RN-60m                 0.0
RN-12H                 0.0
RN-DAY                 0.0
HM                    96.2
PA                  1017.9
PS                  1018.3
TD                     3.2
YY                    2023
MM                      02
DD                      01
HH                      09
MI                      00
Name: 131224, dtype: object

In [ ]:
df['temp'] = 

In [162]:
df.head(10)

,YYMMDDHHMI,STN,WD1,WS1,WDS,WSS,WS10,WS10,TA,RE,...,RN-DAY,HM,PA,PS,TD,YY,MM,DD,HH,MI
0,202302010600,90,312.7,1.9,329.9,2.9,308.2,2.2,4.1,0.0,...,0.0,41.2,1011.5,1013.7,-7.9,2023,02,01,06,00
1,202302010600,92,280.0,3.5,290.0,4.4,270.0,4.6,4.1,-99.9,...,0.0,47.0,1004.3,1013.6,-6.2,2023,02,01,06,00
2,202302010600,93,0.0,0.0,121.0,0.0,152.0,0.5,-6.0,0.0,...,0.0,95.6,1004.1,1016.4,-6.6,2023,02,01,06,00
3,202302010600,95,88.3,0.1,98.5,0.2,41.1,0.2,-3.7,0.0,...,0.0,97.3,997.3,1017.1,-4.1,2023,02,01,06,00
4,202302010600,96,261.6,15.7,263.6,18.1,262.3,15.8,8.9,-99.5,...,0.0,52.7,999.4,1011.4,-0.2,2023,02,01,06,00
5,202302010600,98,341.0,0.0,304.0,0.1,0.0,0.0,-0.4,0.0,...,0.0,95.2,1002.1,1016.6,-1.1,2023,02,01,06,00
6,202302010600,99,0.0,0.0,270.2,0.0,0.0,0.0,-0.7,0.0,...,0.0,99.4,1013.3,1017.2,-0.8,2023,02,01,06,00
7,202302010600,100,274.4,3.8,258.8,5.8,276.0,4.8,-1.5,0.0,...,0.0,71.2,922.1,1015.2,-6.0,2023,02,01,06,00
8,202302010600,101,106.5,0.2,96.6,0.5,105.9,0.6,-4.7,0.0,...,0.0,94.9,1006.0,1015.7,-5.4,2023,02,01,06,00
9,202302010600,102,343.8,4.7,354.6,5.5,346.4,4.8,2.8,0.0,...,0.0,91.5,1011.8,1016.3,1.6,2023,02,01,06,00


# Matching Stations and ids

,지점,지점명
0,116,관악(레)
1,400,강남
2,401,서초
3,402,강동
4,403,송파
5,404,강서
6,405,양천
7,406,도봉
8,407,노원
9,408,동대문


In [1449]:
#####
aws_2023 = pd.read_csv("./dataset/OBS_AWS_TIM_20231011174412.csv", encoding = "EUC-KR")

In [1396]:
aws_402_2018 = pd.read_csv("./dataset/OBS_AWS_TIM_2018_402.csv", encoding = "EUC-KR")
aws_402_2019 = pd.read_csv("./dataset/OBS_AWS_TIM_2019_402.csv")
aws_402_2020 = pd.read_csv("./dataset/SURFACE_AWS_402_HR_2020_2020_2021.csv", encoding = "EUC-KR")
aws_402_2021 = pd.read_csv("./dataset/SURFACE_AWS_402_HR_2021_2021_2022.csv", encoding = "EUC-KR")
aws_402_2022 = pd.read_csv("./dataset/SURFACE_AWS_402_HR_2022_2022_2023.csv", encoding = "EUC-KR")
aws_402_2023 = aws_2023[aws_2023["지점"] == 402]
aws_402 = pd.concat([aws_402_2018, aws_402_2019, aws_402_2020, aws_402_2021, aws_402_2022, aws_402_2023]).reset_index()
aws_402.columns = ["index", "loc", "locname", "time", "temp", "windDir", "windSpd", "precip", "groundP", "sealvlP", "humid", "sunE", "sunT"]
aws_402 = aws_402.drop(columns = ["index", "loc", "locname", "groundP", "sealvlP", "humid", "sunE", "sunT"])

In [1286]:
aws_402

,time,temp,windDir,windSpd,precip
0,2018-01-01 01:00,-1.8,349.5,0.7,0.0
1,2018-01-01 02:00,-2.7,9.4,0.6,0.0
2,2018-01-01 03:00,-3.2,82.7,0.7,0.0
3,2018-01-01 04:00,-4.1,0.0,0.4,0.0
4,2018-01-01 05:00,-4.9,95.2,1.7,0.0
...,...,...,...,...,...
49697,2023-10-09 20:00,17.9,251.6,0.7,0.0
49698,2023-10-09 21:00,17.0,10.0,0.5,0.0
49699,2023-10-09 22:00,15.7,328.3,1.7,0.0
49700,2023-10-09 23:00,14.7,328.4,1.1,0.0


In [1397]:
aws_403_2018 = pd.read_csv("./dataset/OBS_AWS_TIM_2018_403.csv", encoding = "EUC-KR")
aws_403_2019 = pd.read_csv("./dataset/OBS_AWS_TIM_2019_403.csv")
aws_403_2020 = pd.read_csv("./dataset/SURFACE_AWS_403_HR_2020_2020_2021.csv", encoding = "EUC-KR")
aws_403_2021 = pd.read_csv("./dataset/SURFACE_AWS_403_HR_2021_2021_2022.csv", encoding = "EUC-KR")
aws_403_2022 = pd.read_csv("./dataset/SURFACE_AWS_403_HR_2022_2022_2023.csv", encoding = "EUC-KR")
aws_403_2023 = aws_2023[aws_2023["지점"] == 403]
aws_403 = pd.concat([
    aws_403_2018,
    aws_403_2019,
    aws_403_2020,
    aws_403_2021,
    aws_403_2022,
    aws_403_2023]).reset_index()
aws_403.columns = ["index", "loc", "locname", "time", "temp", "windDir", "windSpd", "precip", "groundP", "sealvlP", "humid", "sunE", "sunT"]
aws_403 = aws_403.drop(columns = ["index", "loc", "locname", "groundP", "sealvlP", "humid", "sunE", "sunT"])

In [1398]:
aws_413_2018 = pd.read_csv("./dataset/OBS_AWS_TIM_2018_413.csv", encoding = "EUC-KR")
aws_413_2019 = pd.read_csv("./dataset/OBS_AWS_TIM_2019_413.csv")
aws_413_2020 = pd.read_csv("./dataset/SURFACE_AWS_413_HR_2020_2020_2021.csv", encoding = "EUC-KR")
aws_413_2021 = pd.read_csv("./dataset/SURFACE_AWS_413_HR_2021_2021_2022.csv", encoding = "EUC-KR")
aws_413_2022 = pd.read_csv("./dataset/SURFACE_AWS_413_HR_2022_2022_2023.csv", encoding = "EUC-KR")
aws_413_2023 = aws_2023[aws_2023["지점"] == 413]
aws_413 = pd.concat([
    aws_413_2018,
    aws_413_2019,
    aws_413_2020,
    aws_413_2021,
    aws_413_2022,
    aws_413_2023]).reset_index()
aws_413.columns = ["index", "loc", "locname", "time", "temp", "windDir", "windSpd", "precip", "groundP", "sealvlP", "humid", "sunE", "sunT"]
aws_413 = aws_413.drop(columns = ["index", "loc", "locname", "groundP", "sealvlP", "humid", "sunE", "sunT"])

In [1399]:
aws_415_2018 = pd.read_csv("./dataset/OBS_AWS_TIM_2018_415.csv", encoding = "EUC-KR")
aws_415_2019 = pd.read_csv("./dataset/OBS_AWS_TIM_2019_415.csv")
aws_415_2020 = pd.read_csv("./dataset/SURFACE_AWS_415_HR_2020_2020_2021.csv", encoding = "EUC-KR")
aws_415_2021 = pd.read_csv("./dataset/SURFACE_AWS_415_HR_2021_2021_2022.csv", encoding = "EUC-KR")
aws_415_2022 = pd.read_csv("./dataset/SURFACE_AWS_415_HR_2022_2022_2023.csv", encoding = "EUC-KR")
aws_415_2023 = aws_2023[aws_2023["지점"] == 415]
aws_415 = pd.concat([
    aws_415_2018,
    aws_415_2019,
    aws_415_2020,
    aws_415_2021,
    aws_415_2022,
    aws_415_2023]).reset_index()
aws_415.columns = ["index", "loc", "locname", "time", "temp", "windDir", "windSpd", "precip", "groundP", "sealvlP", "humid", "sunE", "sunT"]
aws_415 = aws_415.drop(columns = ["index", "loc", "locname", "groundP", "sealvlP", "humid", "sunE", "sunT"])

In [1516]:
aws_421_2018 = pd.read_csv("./dataset/OBS_AWS_TIM_2018_421.csv", encoding = "EUC-KR")
aws_421_2019 = pd.read_csv("./dataset/OBS_AWS_TIM_2019_421.csv")
aws_421_2020 = pd.read_csv("./dataset/SURFACE_AWS_421_HR_2020_2020_2021.csv", encoding = "EUC-KR")
aws_421_2021 = pd.read_csv("./dataset/SURFACE_AWS_421_HR_2021_2021_2022.csv", encoding = "EUC-KR")
aws_421_2022 = pd.read_csv("./dataset/SURFACE_AWS_421_HR_2022_2022_2023.csv", encoding = "EUC-KR")
aws_421_2023 = aws_2023[aws_2023["지점"] == 421]
aws_421 = pd.concat([
    aws_421_2018,
    aws_421_2019,
    aws_421_2020,
    aws_421_2021,
    aws_421_2022,
    aws_421_2023]).reset_index()
aws_421.columns = ["index", "loc", "locname", "time", "temp", "windDir", "windSpd", "precip", "groundP", "sealvlP", "humid", "sunE", "sunT"]
aws_421 = aws_421.drop(columns = ["index", "loc", "locname", "groundP", "sealvlP", "humid", "sunE", "sunT"])

In [1517]:
aws_421

,time,temp,windDir,windSpd,precip
0,2018-01-01 01:00,-0.2,310.2,1.7,0.0
1,2018-01-01 02:00,-0.4,260.2,1.5,0.0
2,2018-01-01 03:00,-0.5,266.7,2.0,0.0
3,2018-01-01 04:00,-0.8,266.6,2.1,0.0
4,2018-01-01 05:00,-0.8,271.3,1.8,0.0
...,...,...,...,...,...
49625,2023-10-09 20:00,18.2,241.8,3.9,0.0
49626,2023-10-09 21:00,17.5,296.7,1.7,0.0
49627,2023-10-09 22:00,16.1,32.1,1.4,0.5
49628,2023-10-09 23:00,15.5,39.2,2.1,0.0


In [1401]:
aws_510_2018 = pd.read_csv("./dataset/OBS_AWS_TIM_2018_510.csv", encoding = "EUC-KR")
aws_510_2019 = pd.read_csv("./dataset/OBS_AWS_TIM_2019_510.csv")
aws_510_2020 = pd.read_csv("./dataset/SURFACE_AWS_510_HR_2020_2020_2021.csv", encoding = "EUC-KR")
aws_510_2021 = pd.read_csv("./dataset/SURFACE_AWS_510_HR_2021_2021_2022.csv", encoding = "EUC-KR")
aws_510_2022 = pd.read_csv("./dataset/SURFACE_AWS_510_HR_2022_2022_2023.csv", encoding = "EUC-KR")
aws_510_2023 = aws_2023[aws_2023["지점"] == 510]
aws_510 = pd.concat([
    aws_510_2018,
    aws_510_2019,
    aws_510_2020,
    aws_510_2021,
    aws_510_2022,
    aws_510_2023]).reset_index()
aws_510.columns = ["index", "loc", "locname", "time", "temp", "windDir", "windSpd", "precip", "groundP", "sealvlP", "humid", "sunE", "sunT"]
aws_510 = aws_510.drop(columns = ["index", "loc", "locname", "groundP", "sealvlP", "humid", "sunE", "sunT"])

In [1402]:
aws_889_2018 = pd.read_csv("./dataset/OBS_AWS_TIM_2018_889.csv", encoding = "EUC-KR")
aws_889_2019 = pd.read_csv("./dataset/OBS_AWS_TIM_2019_889.csv")
aws_889_2020 = pd.read_csv("./dataset/SURFACE_AWS_889_HR_2020_2020_2021.csv", encoding = "EUC-KR")
aws_889_2021 = pd.read_csv("./dataset/SURFACE_AWS_889_HR_2021_2021_2022.csv", encoding = "EUC-KR")
aws_889_2022 = pd.read_csv("./dataset/SURFACE_AWS_889_HR_2022_2022_2023.csv", encoding = "EUC-KR")
aws_889_2023 = aws_2023[aws_2023["지점"] == 889]
aws_889 = pd.concat([
    aws_889_2018,
    aws_889_2019,
    aws_889_2020,
    aws_889_2021,
    aws_889_2022,
    aws_889_2023]).reset_index()
aws_889.columns = ["index", "loc", "locname", "time", "temp", "windDir", "windSpd", "precip", "groundP", "sealvlP", "humid", "sunE", "sunT"]
aws_889 = aws_889.drop(columns = ["index", "loc", "locname", "groundP", "sealvlP", "humid", "sunE", "sunT"])

In [1293]:
aws_402

,time,temp,windDir,windSpd,precip
0,2018-01-01 01:00,-1.8,349.5,0.7,0.0
1,2018-01-01 02:00,-2.7,9.4,0.6,0.0
2,2018-01-01 03:00,-3.2,82.7,0.7,0.0
3,2018-01-01 04:00,-4.1,0.0,0.4,0.0
4,2018-01-01 05:00,-4.9,95.2,1.7,0.0
...,...,...,...,...,...
49697,2023-10-09 20:00,17.9,251.6,0.7,0.0
49698,2023-10-09 21:00,17.0,10.0,0.5,0.0
49699,2023-10-09 22:00,15.7,328.3,1.7,0.0
49700,2023-10-09 23:00,14.7,328.4,1.1,0.0


In [1403]:
def full_time(climate_data):
    climate_data['time'] = pd.to_datetime(climate_data['time'], format='%Y-%m-%d %H:%M')
    time_indx = pd.date_range("2018-01-01 1:00", "2023-10-10 00:00", freq = "H")
    tmp_time = pd.DataFrame({"time": time_indx})
    new_climate_data = pd.merge(tmp_time, climate_data, left_on = "time", right_on = "time", how = "left")
    return new_climate_data

In [1518]:
aws_402 = full_time(aws_402)
aws_403 = full_time(aws_403)
aws_413 = full_time(aws_413)
aws_415 = full_time(aws_415)
aws_421 = full_time(aws_421)
aws_510 = full_time(aws_510)
aws_889 = full_time(aws_889)

In [1454]:
aws_421

,time,index,loc,locname,temp,windDir,windSpd,precip,groundP,sealvlP,humid,sunE,sunT
0,2018-01-01 01:00:00,0.0,421.0,성동,-0.2,310.2,1.7,0.0,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 02:00:00,1.0,421.0,성동,-0.4,260.2,1.5,0.0,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 03:00:00,2.0,421.0,성동,-0.5,266.7,2.0,0.0,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 04:00:00,3.0,421.0,성동,-0.8,266.6,2.1,0.0,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 05:00:00,4.0,421.0,성동,-0.8,271.3,1.8,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50587,2023-10-09 20:00:00,27067.0,421.0,성동,18.2,241.8,3.9,0.0,1015.2,1019.3,65.0,NaN,NaN
50588,2023-10-09 21:00:00,27068.0,421.0,성동,17.5,296.7,1.7,0.0,1015.5,1019.6,63.0,NaN,NaN
50589,2023-10-09 22:00:00,27069.0,421.0,성동,16.1,32.1,1.4,0.5,1015.9,1020.0,72.0,NaN,NaN
50590,2023-10-09 23:00:00,27070.0,421.0,성동,15.5,39.2,2.1,0.0,1015.8,1020.0,71.0,NaN,NaN


In [1532]:
aws_402.to_csv("./dataset/climate_data_402.csv", index = False)
aws_403.to_csv("./dataset/climate_data_403.csv", index = False)
aws_413.to_csv("./dataset/climate_data_413.csv", index = False)
aws_415.to_csv("./dataset/climate_data_415.csv", index = False)
aws_421.to_csv("./dataset/climate_data_421.csv", index = False)
aws_510.to_csv("./dataset/climate_data_510.csv", index = False)
aws_889.to_csv("./dataset/climate_data_889.csv", index = False)

---

In [45]:
colnames_waterLvl = ["date"]
for i in range(1, 25):
    colnames_waterLvl.append("lvl%02d"%i)

In [46]:
lvl_1018640 = pd.read_csv("./dataset/수위_시자료_서울시(광진교)_1018640.csv", header = 0, names = colnames_waterLvl)
lvl_1018680 = pd.read_csv("./dataset/수위_시자료_서울시(잠수교)_1018680.csv", header = 0, names = colnames_waterLvl)
lvl_1018662 = pd.read_csv("./dataset/수위_시자료_서울시(청담대교)_1018662.csv", header = 0, names = colnames_waterLvl)
lvl_1018683 = pd.read_csv("./dataset/수위_시자료_서울시(한강대교)_1018683.csv", header = 0, names = colnames_waterLvl)

In [1419]:
lvl_1018640

,date,lvl01,lvl02,lvl03,lvl04,lvl05,lvl06,lvl07,lvl08,lvl09,...,lvl15,lvl16,lvl17,lvl18,lvl19,lvl20,lvl21,lvl22,lvl23,lvl24
0,2023-10-09,0.54,0.55,0.56,0.56,0.56,0.57,0.58,0.58,0.58,...,0.59,0.60,0.61,0.62,0.60,0.58,0.58,0.57,0.52,0.51
1,2023-10-08,0.52,0.52,0.51,0.53,0.54,0.54,0.55,0.56,0.57,...,0.62,0.61,0.58,0.57,0.55,0.54,0.54,0.53,0.52,0.53
2,2023-10-07,0.52,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.56,...,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.57,0.57
3,2023-10-06,0.54,0.54,0.53,0.53,0.53,0.53,0.56,0.57,0.58,...,0.60,0.58,0.56,0.54,0.52,0.51,0.51,0.51,0.51,0.51
4,2023-10-05,0.49,0.49,0.49,0.49,0.49,0.49,0.50,0.53,0.57,...,0.54,0.56,0.58,0.60,0.60,0.59,0.55,0.54,0.52,0.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,2018-01-05,1.03,1.03,1.03,1.03,1.03,1.03,1.03,1.03,1.03,...,1.02,1.02,1.02,1.02,1.02,1.02,1.02,1.02,1.02,1.02
2104,2018-01-04,1.06,1.06,1.06,1.05,1.05,1.04,1.04,1.03,1.03,...,1.03,1.03,1.03,1.03,1.03,1.03,1.03,1.03,1.03,1.03
2105,2018-01-03,1.02,1.02,1.03,1.03,1.03,1.03,1.04,1.04,1.05,...,1.05,1.05,1.06,1.06,1.06,1.06,1.06,1.06,1.06,1.06
2106,2018-01-02,1.03,1.03,1.03,1.03,1.03,1.03,1.03,1.03,1.02,...,1.01,1.02,1.02,1.02,1.02,1.02,1.02,1.02,1.02,1.02


In [1420]:
# lvl_1018640 = lvl_1018640.iloc[:282]
# lvl_1018680 = lvl_1018680.iloc[:282]
# lvl_1018662 = lvl_1018662.iloc[:282]
# lvl_1018683 = lvl_1018683.iloc[:282]

In [47]:
def lvlchformat(water_level):
    water_level = pd.wide_to_long(water_level, ["lvl"], i = "date", j = "hours")
    water_level.sort_values(by = ["date", "hours"], axis = 0, inplace = True)
    water_level.reset_index(drop = False, inplace = True)
    water_level['hours'].loc[water_level['hours'] == 24] = 0 #change 24h to 0h
    water_level['date'] = pd.to_datetime(water_level['date'], format='%Y-%m-%d')
    water_level['date'].loc[water_level['hours'] == 0] += timedelta(days = 1)
    water_level["time"] = water_level["date"].astype(str) + " " + water_level["hours"].astype(str) + ":00"
    water_level['time'] = pd.to_datetime(water_level['time'], format='%Y-%m-%d %H:%M')
    water_level = water_level[:-1]
    water_level = water_level.iloc[:, 2:]
    
    return water_level

In [48]:
lvl_1018640 = lvlchformat(lvl_1018640)
lvl_1018680 = lvlchformat(lvl_1018680)
lvl_1018662 = lvlchformat(lvl_1018662)
lvl_1018683 = lvlchformat(lvl_1018683)

/tmp/ipykernel_17709/423137950.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water_level['hours'].loc[water_level['hours'] == 24] = 0 #change 24h to 0h
/tmp/ipykernel_17709/423137950.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water_level['date'].loc[water_level['hours'] == 0] += timedelta(days = 1)
/tmp/ipykernel_17709/423137950.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water_level['hours'].loc[water_level['hou

In [52]:
def full_time2(climate_data):
    climate_data['time'] = pd.to_datetime(climate_data['time'], format='%Y-%m-%d %H:%M')
    time_indx = pd.date_range("2018-01-01 1:00", "2023-10-09 23:00", freq = "H")
    tmp_time = pd.DataFrame({"time": time_indx})
    new_climate_data = pd.merge(climate_data, tmp_time, left_on = "time", right_on = "time", how = "right")
    return new_climate_data

In [53]:
lvl_1018640 = full_time2(lvl_1018640)
lvl_1018680 = full_time2(lvl_1018680)
lvl_1018662 = full_time2(lvl_1018662)
lvl_1018683 = full_time2(lvl_1018683)

In [54]:
lvl_1018683

,lvl,time
0,0.78,2018-01-01 01:00:00
1,0.75,2018-01-01 02:00:00
2,0.73,2018-01-01 03:00:00
3,0.70,2018-01-01 04:00:00
4,0.69,2018-01-01 05:00:00
...,...,...
50586,0.53,2023-10-09 19:00:00
50587,0.55,2023-10-09 20:00:00
50588,0.57,2023-10-09 21:00:00
50589,0.58,2023-10-09 22:00:00


In [96]:
lvl_1018640.to_csv("./dataset/lvl_data_1018640.csv", index = False)
lvl_1018680.to_csv("./dataset/lvl_data_1018680.csv", index = False)
lvl_1018662.to_csv("./dataset/lvl_data_1018662.csv", index = False)
lvl_1018683.to_csv("./dataset/lvl_data_1018683.csv", index = False)

In [6]:
import numpy as np

In [4]:
a = np.array([[[-0.96966253],
        [-0.96969018],
        [-0.96971831]],

       [[-0.96969018],
        [-0.96971831],
        [-0.96974693]],

       [[-0.96971831],
        [-0.96974693],
        [-0.96977604]]])

In [28]:
b = np.array([[[-1.08109187],
        [-1.08256533],
        [-1.08403912]],

       [[20],
        [-1.08403912],
        [-1.08551323]],

       [[-1.08403912],
        [-1.08551323],
        [10]]])

In [29]:
c = np.concatenate([a, b], axis = 2)

In [30]:
c

array([[[-0.96966253, -1.08109187],
        [-0.96969018, -1.08256533],
        [-0.96971831, -1.08403912]],

       [[-0.96969018, 20.        ],
        [-0.96971831, -1.08403912],
        [-0.96974693, -1.08551323]],

       [[-0.96971831, -1.08403912],
        [-0.96974693, -1.08551323],
        [-0.96977604, 10.        ]]])

In [31]:
my_sum = np.sum(c, axis=2)

In [32]:
tr = np.transpose(my_sum)

In [33]:
tr

array([[-2.0507544 , 19.03030982, -2.05375743],
       [-2.05225551, -2.05375743, -2.05526016],
       [-2.05375743, -2.05526016,  9.03022396]])

In [34]:
my_sum

array([[-2.0507544 , -2.05225551, -2.05375743],
       [19.03030982, -2.05375743, -2.05526016],
       [-2.05375743, -2.05526016,  9.03022396]])

In [35]:
tr[0] # 1시간 후

array([-2.0507544 , 19.03030982, -2.05375743])

In [36]:
tr[1] # 2시간 후

array([-2.05225551, -2.05375743, -2.05526016])

In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame([[1,2], [3,4]])

In [3]:
df

,0,1
0,1,2
1,3,4


In [7]:
df = np.array(df)

In [8]:
df

array([[1, 2],
       [3, 4]])